In [ ]:
# Setting html stuff for the rest of the notebook
from IPython.core.display import display, HTML, Javascript
html_contents ="""
<!DOCTYPE html>
<html lang="en">
    <head>
    <style>
    
    .top_section{
        background-color: #a13d2d;
        color: black;
        font-family: Copperplate, Papyrus, fantasy;
        font-weight: 800;
        font-size: 25px;
        padding: 8px 14px;
        margin-bottom: 6px;
    }
    
    .section_title{
        background-color: #FF8C00;
        color: black;
        font-family: Copperplate, Papyrus, fantasy;
        font-weight: 600;
        font-size: 25px;
        padding: 6px 12px;
        margin-bottom: 5px;
    }
    .subsection_title{
        background: #FFD580;
        font-weight: 400;
        color: black;
        font-family: Copperplate, Papyrus, fantasy;
        font-size: 21px;
        padding: 6px 12px;
        margin-bottom: 0px;
    }
    .subsection_detail{
        background: #D3D3D3;
        font-weight: 300;
        color: black;
        font-family: Copperplate, Papyrus, fantasy;
        font-size: 18px;
        border: 1px solid #d7d7d7;
        padding: 1px 10px 2px;
        box-shadow: 1px 1px 2px 1px rgba(0,0,0,0.3);
        margin-bottom: 3px;
    }
    </style>
    </head>
    
</html>
"""

HTML(html_contents)

# <div class="top_section">1. Introduction</div>
- Hello fellow kagglers! In this notebook we're going to analyse the latest <span style="color:#00FFFF;"> kaggle </span>  user survey and by doing so we're going to get some insights.
- I'm planning to go through feature(question) by feature and take closer look those features to inspect their relationships with previous features. So it's gonna take a while to finish this notebook with all features involved. I'm planning to update it regulary whenever I have free time, I hope you like it!

- This notebook will consist of two main parts:
    - First part will be about a general look of a participant, we are going to find about an averege <span style="color:#00FFFF;"> kaggle </span> user.
    - In second part I plan to guide through more specialized parts of a data science with a randomly selected person from these instances we inspected in first part, this randomly selected imaginary character will be named <span style="color:#FF0000;;font-weight:bold"> Robert </span> don't ask why yet :) you'll get to know about him in second part and we will try to find best path for an aspiring data scientist...


- Let's get started...

# <div class="section_title">2. The Data</div>
<img src="https://i.imgur.com/UlXkI3O.png" style="height: 180; width: 100%; object-fit: cover;"/>

- It all begins with the data, without it we can't tell the story... Imagine being a medieval bard, he needs to travel around to listen common folk stories, tavern by tavern, village by village and then he creates an epic to tell the tale. In essence we do similar thing, maybe it's not as cool as bards... Oh well...
- Altough our stories might not be that epic we still can create some cool stuff with the help of some items in our toolbelt.
- Finding and creating meaningful relationships between data points where people can't see at first look sounds like some kind of magic. With the right data and tools we can even forecast the future! How do you like them apples bards! 
- So story telling from the data what makes us unique in a way and that's what I'm going to try in rest of this notebook. We have the data, we have the tools and we have story to tell!

In [ ]:
# importing needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode

from scipy import stats
from scipy.stats import norm

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

import re

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

# setting some globl config
plt.style.use('fivethirtyeight')
cust_color = ['#fdc029',
'#f7c14c',
'#f0c268',
'#e8c381',
'#dfc498',
'#d4c5af',
'#c6c6c6',
'#a6a6a8',
'#86868a',
'#68686d',
'#4b4c52',
'#303138',
'#171820',
]
plt.rcParams['figure.figsize'] = (18,14)
plt.rcParams['figure.dpi'] = 300
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = cust_color[3]
plt.rcParams["grid.alpha"] = 0.5
plt.rcParams["grid.linestyle"] = '--'
plt.rcParams["font.family"] = "monospace"

plt.rcParams['axes.edgecolor'] = 'black'
plt.rcParams['figure.frameon'] = True
plt.rcParams['axes.spines.left'] = True
plt.rcParams['axes.spines.bottom'] = True
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.linewidth'] = 1.5

In [ ]:
def one_hot_count(df: pd.DataFrame, values:list,text:str, colors=None, shift_text=200, x_freq=2000, title=None, fontsize=12):
    """A function for plotting multi choice questions."""
    
    df2= df.copy()
    for val in values:
        df2[text+val]=df2[text+val].fillna('unk')
        df2[text+val]=df2[text+val].apply(lambda x: x.strip())
        df2[text+val]=df2[text+val].map({val:1, 'unk':0})
    df2.columns = df2.columns.str.replace(text, '', regex=False)
    
    counts = {}
    for col in values:
        counts[col]=df2[col].sum()
    counts = pd.DataFrame.from_dict(counts, orient='index')
    
    fig, ax = plt.subplots(figsize=(18, 14))
    counts = counts.rename(columns={0: 'Count'}).sort_values(by='Count',ascending=False)
    sns.barplot(y=counts.index,
                    x=counts['Count'], palette=colors, edgecolor='black', saturation = 1.5, linewidth = 1.5)
    for n, i in enumerate(counts['Count']):    
        ax.text(counts['Count'][n]+shift_text, 
                n, #Y location
                s=f'{round(counts.Count[n]/len(df2)*100,1)}%', 
                va='center', 
                ha='right', 
                color='white', 
                fontsize=fontsize,
                bbox=dict(boxstyle='round',facecolor=cust_color[-1], alpha=0.5))
    plt.xticks(np.arange(x_freq, len(df2)+1, x_freq))
    plt.title(title, fontname = 'monospace', weight='bold')
    plt.show()
    return df2

In [ ]:
# Reading the Data
df = pd.read_csv('../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv')

# Setting New Column
df.columns = df.iloc[0]
df = df[1:]

# clearing some column stuff
df.columns = df.columns.str.replace(' - Selected Choice', '')
df.columns = df.columns.str.replace(r'\(Select all that apply\) -', '')
df.columns = df.columns.str.replace(' (Select all that apply): -', '', regex=False)
df.rename(columns=lambda x: re.sub(' +', ' ', x), inplace=True)
df.columns = df.columns.str.rstrip()

* Ok we just geared our toolbelt and loaded the dataset, let's inspect it column by column to see if we can come up with something...

# <div class="section_title">2.1 Survey Duration</div>

- We do have the statistics of how much time (in seconds) has user been interacting with the questions and answering them. Maybe we can find some interesting stuff?
- A good rule of thumb is to aim for a survey takes no more than 15 to 20 minutes to complete, usually shorter than that.
- Let's take a look some basic aggregations to see if that's the case with <span style="color:#00FFFF;"> kaggle </span> survey too.

In [ ]:
# changing dtype to in for numerical aggregations
df['Duration (in seconds)']=df['Duration (in seconds)'].astype(int)
mx=df['Duration (in seconds)'].max()
print(f'Maximum Survey Time: {mx} seconds.')

- It looks like we have some outliers here, because an user spent almost 700 hours on the survey! I mean he/she can be really indecisive but let's be real here, it probably caused by a system error or maybe someone took a reaaaaaly long coffee break, we all need some rest, right?..
- To get rid of outliers I'm going to clip values bigger than 10000 seconds to see if it gets better.

In [ ]:
def plot_dist3(df, feature, title, clip_value):
    
    # Creating a customized chart. and giving in figsize and everything.
    
    fig = plt.figure(constrained_layout=True)
    
    # creating a grid of 3 cols and 3 rows.
    
    grid = gridspec.GridSpec(ncols=3, nrows=2, figure=fig)

    # Customizing the histogram grid.
    
    ax1 = fig.add_subplot(grid[0, :2])
    
    # Set the title.
    
    ax1.set_title('Histogram')
    
    # plot the histogram.
    
    sns.distplot(df.loc[:, feature].clip(0,clip_value),
                 hist=True,
                 kde=True,
                 fit=norm,
                  hist_kws={
                 'rwidth': 0.85,
                 'edgecolor': 'black',
                 'linewidth':.5,
                 'alpha': 0.8},
                 ax=ax1,
                 color=cust_color[0])
    
    ax1.axvline(df.loc[:, feature].clip(0,clip_value).mean(), color='Green', linestyle='dashed', linewidth=3)

    min_ylim, max_ylim = plt.ylim()
    ax1.text(df.loc[:, feature].clip(0,clip_value).mean()*1.25, max_ylim*0.85, 'Mean: {:.2f}'.format(df.loc[:, feature].clip(0,clip_value).mean()), color='Green', fontsize='12',
             bbox=dict(boxstyle='round',facecolor='red', alpha=0.5))
    ax1.legend(labels=['Actual','Normal'])
    ax1.xaxis.set_major_locator(MaxNLocator(nbins=12))
    
    
    ax1.annotate(
    # Label and coordinate
    'Unexpected Spike here!', xy=(10000, 0.00025),xytext=(10500, 0.0004) ,
    horizontalalignment="center",
    # Custom arrow
    arrowprops=dict(arrowstyle='simple',lw=1, color='black'), fontsize=8
    )

    # customizing the QQ_plot.
    
    ax2 = fig.add_subplot(grid[1, :2])
    
    # Set the title.
    
    ax2.set_title('Probability Plot')
    
    # Plotting the QQ_Plot.
    stats.probplot(df.loc[:, feature].clip(0,clip_value),
                   plot=ax2)
    ax2.get_lines()[0].set_markerfacecolor('#e74c3c')
    ax2.get_lines()[0].set_markersize(12.0)
    ax2.xaxis.set_major_locator(MaxNLocator(nbins=16))

    # Customizing the Box Plot:
    
    ax3 = fig.add_subplot(grid[:, 2])
    # Set title.
    
    ax3.set_title('Box Plot')
    
    # Plotting the box plot.
    
    sns.boxplot(y=feature, data=df, ax=ax3, color=cust_color[0])
    ax3.yaxis.set_major_locator(MaxNLocator(nbins=24))
    ax3.set_ylim(0,clip_value)

    plt.suptitle(f'{title}', fontsize=24, fontname = 'monospace', weight='bold')

In [ ]:
plot_dist3(df, 'Duration (in seconds)', 'Survey Duration Distribution', 10000)

- Looks much better! We can see that most of the users finished their answering under 20 minutes, which fits to rule of thumb I mentioned above. We still have some outliers but they look more humane levels to me...
- On average, the survey took around 20 minutes where most of the users were done in 10-15 minutes...
- By looking at the probability plot we can see that data doesn't follow the normal distribution. I know, I know you can already see that in histogram too :)

# <div class="section_title">2.2 Demographics</div>

- Thanks to <span style="color:#00FFFF;"> kaggle </span> for being available across the world easily and free we do have great demographic data to explore and that's what we're going to in next chapter...

# <div class="subsection_title">2.2.1 Age</div>
- In this part of the data we have age groups of participants,
- By looking at recent years trends and Data Science being labelled as one of the best jobs by some news sources I expect high number of young participants.
- With the all hype around data and data related jobs I expect increased demand from employers and graduates with interest in <span style="color:#00FFFF;"> kaggle </span>.
- Let's see if data confirms my hypothesis...

In [ ]:
# plotting and styling
fig, ax = plt.subplots(1, 2, figsize=(12,8))
sns.barplot(x=df['What is your age (# years)?'].value_counts().index, y=df['What is your age (# years)?'].value_counts().values, palette=cust_color, ax=ax[0],
           edgecolor='black', linewidth=1.5, saturation=1.5)
ax[0].yaxis.set_major_locator(MaxNLocator(nbins=20))
ax[0].tick_params(axis='x', which='major', labelsize=10)
ax[0].set_ylabel('Count', weight='semibold', fontname = 'monospace')
ax[0].set_xlabel('Age Group', weight='semibold', fontname = 'monospace')



ax[1].pie(df['What is your age (# years)?'].value_counts(), labels = df['What is your age (# years)?'].value_counts().index, colors = cust_color, autopct='%1.1f%%',
        explode=[0.03 for i in df['What is your age (# years)?'].value_counts().index])


plt.suptitle('Age Distribution of Kaggle Users', fontname = 'monospace', weight='bold')
plt.show()

- Please don't burn my plot to the ground because of the "Pie Chart", I know many people hate them, but I find them useful for small categories like this.
- Ok with leaving pie chart hate behind let's check the values we got, it seems my first instinct was true; most of the participants are younger than 35 years old.
- And we can also see that 18-21 age group almost made it to top (0.1% difference), so the trend for data related jobs still going strong among students.
- By looking at the other side of the chart we see either really experienced users or people got interest in data science in late stages of their career, believe me it's never too late! It's nice to see couple hundred of them hanging around...

# <div class="subsection_detail">2.2.1.1 Survey Duration vs. Age</div>
- Maybe it's not that important to check this relationship but I got the curiosity to see if there is a meaningful relation between these two...

In [ ]:
# eliminating outliers
z=df[df['Duration (in seconds)']<10000].groupby('What is your age (# years)?')['Duration (in seconds)'].mean().sort_values(ascending=False)
# plotting and styling
fig, ax = plt.subplots(figsize=(12,4))
sns.barplot(x=z.index, y=z.values, palette=cust_color, edgecolor='black', linewidth=1.5, saturation=1.5)

plt.xlabel("Age Group", fontname = 'monospace', weight='semibold')
plt.ylabel("Mean Time to Finish Survey", fontname = 'monospace', weight='semibold')
plt.title('Average Time to Finish Survey by Age', fontname = 'monospace', weight='bold')
plt.xticks(fontsize=12)
plt.yticks(np.arange(0, z.values.max()+1, 100))
plt.show()

- It looks like younger people tend to finish survey a little bit faster, or maybe experienced people are paying attention to details a little bit more, anyways no need to hurry eh? :) Let's get on the next demographic feature...


# <div class="subsection_title">2.2.2 Gender</div>
- While Data Science sector growing fast, the male dominance still stands, that would be worrying if the trend keeps going incoming years. For some of the people it might not sound like a obvious problem but incresed bias in data related works might affect the outcomes of machine learning/AI predictions in near future. Let's take a look how're things stand for this subject on <span style="color:#00FFFF;"> kaggle </span>...

In [ ]:
# plotting and styling
fig, ax = plt.subplots(figsize=(12,4))
sns.barplot(x=df['What is your gender?'].value_counts().index, y=df['What is your gender?'].value_counts().values, palette=cust_color[::4], edgecolor='black', linewidth=1.5, saturation=1.5)
plt.xlabel("# Users", fontname = 'monospace', weight='semibold')
plt.ylabel("Gender", fontname = 'monospace', weight='semibold')
plt.title('Gender Distribution of Kaggle Users', fontname = 'monospace', weight='bold');

- Uhmm doesn't look much better than last year if I recall correctly, they stand around same like last year (~78%, ~19%). Maybe looking at some other relations with this data might help us to get better insights.

# <div class="subsection_detail">2.2.2.1 Age vs Gender</div>

In [ ]:
# Plotting interactive sunburst:

fig = px.sunburst(data_frame=df,
                  path=['What is your gender?', 'What is your age (# years)?'],
                  color='What is your gender?',
                  color_discrete_sequence=cust_color[::-4],
                  title='Gender and Age'
                 )

fig.update_traces(textinfo='label+percent parent')
fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
fig.show()

- Another Pie'ish looking chart right? I'm sorry but this one looks little bit cooler, wouldn't you agree :) Anyways;
- This one gives us a little bit better insight, by looking at age groups of dominant genders we can see that number of younger women are getting into data science/machine learning area a bit more than their counterparts.
- 18-21 age group takes 5% more in their gender group than males, again 22-24 is 2% more than men followed by 25-29 which again little bit bigger fraction in women than men.
- So by looking at percentages we would say younger women are getting into data science more and more, it's nice to have diversity among users in future!

In [ ]:
def count_dist(df, colname=None, fixlabel=False, f_axis=None, fixlabel_n=None, fixlabel_txt=None, max_idx=30, fontsize=12, palette=cust_color, rotation=45,
              title='X distribution of Kaggle Users', y_label='', shift=-0.005):
    """A function for counting and displaying categorical variables including percentage texts."""
    fig, ax = plt.subplots()
    sns.barplot(y=df[colname].value_counts().index[:max_idx],
                x=df[colname].value_counts().values[:max_idx], palette=palette, edgecolor='black', linewidth=1.5, saturation = 1.5)
    z=df[colname].value_counts().values[:max_idx]
    for n, i in enumerate(df[colname].value_counts().index[:max_idx]):    
        ax.text(df[colname].value_counts().values[:max_idx][n]+shift, 
                n, #Y location
                s=f'{round(z[n]/df.shape[0]*100,1)}%',                 
                va='center', 
                ha='right', 
                color='white', 
                fontsize=fontsize,
                bbox=dict(boxstyle='round',facecolor='black', alpha=0.5))
    if fixlabel:
        if f_axis == 'x':
            labels = [item.get_text() for item in ax.get_xticklabels()]
            labels[fixlabel_n] = fixlabel_txt
            ax.set_xticklabels(labels)
        else:
            labels = [item.get_text() for item in ax.get_yticklabels()]
            labels[fixlabel_n] = fixlabel_txt
            ax.set_yticklabels(labels)            

    plt.title(title, fontname = 'monospace', weight='bold')
    del z
    
    plt.yticks(fontsize=12,rotation=rotation)
    plt.xlabel("# Users", fontname = 'monospace', weight='semibold')
    plt.ylabel(y_label, fontname = 'monospace', weight='semibold')
    plt.show()

# <div class="subsection_title">2.2.3 Countries</div>
- Another interesting demographic statistic for me is the country where participant currently residing. Again thanks to <span style="color:#00FFFF;"> kaggle </span>'s ease to access and global availability we have chance to see how's the data science community develops around the world.
- For me it's important to see equal growth around the world to build stronger community and wealth.
- First let's take a look top countries with highest number of participants:

In [ ]:
count_dist(df=df, colname='In which country do you currently reside?', max_idx=12, fontsize=16, rotation=45, title='Country Distribution of Kaggle Users for Top Countries',
           shift=400, y_label = 'Country')

- Maybe plotting these values on World Map would be more visually satisfying and allows us to see wider perspective:

In [ ]:
# plotting choropleth
import plotly.offline as py
py.offline.init_notebook_mode()

def plot_in_map(locations,counts,title):
    data = [ dict(
            type = 'choropleth',
            locations = locations,
            z = counts,
            locationmode = 'country names',
            autocolorscale = True,
            marker = dict(
                line = dict(color = 'rgb(58,100,69)', width = 0.6)),
                colorbar = dict(autotick = True, tickprefix = '', title = '# of Kagglers')
                )
           ]

    layout = dict(
        title = title,
        geo = dict(
            showframe = True,
            showcoastlines = True,
            showocean = True,
            showlakes = True,
            oceancolor = '#00008B',
            projection = dict(
            type = 'robinson'
            ),
        margin = dict(b = 0, t = 0, l = 0, r = 0,pad= 5)
                ),
        autosize = True
        )

    fig = dict(data=data, layout=layout)
    
    py.iplot(fig, validate=False, filename='world-map')
z = df['In which country do you currently reside?'].value_counts()
plot_in_map(locations=z.index,
            counts=z.values,
            title='Participants Around the World')

- Looks like most of the participants are from Asia, led by India which keeps top spot in worldwide too.
- In Africa; Nigeria is the flag bearer of the number of participants, followed by Egypt.
- North America is led by USA which also holding global second place.
- In South America Brazil has the biggest community then comes the Colombia.
- Europe looks much more balanced than rest of the world if we don't count Austuralia (continent) :)

# <div class="subsection_detail">2.2.3.1 Age vs. Country</div>
- Here we're going to check age group frequencies of the given countries.
- It can give us insights about where's the data science community grows with lots of young people getting interested in the field.

In [ ]:
# getting percentages and plotting them in heatmap
z=df.groupby([ 'In which country do you currently reside?', 'What is your age (# years)?']).size().unstack().fillna(0).astype('int16')
fig, ax = plt.subplots(figsize=(14, 24))
sns.heatmap(z.apply(lambda x: x/x.sum(), axis=1), xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.3f', cbar=False)
plt.title('Age Distribution by Country', fontname = 'monospace', weight='bold')
plt.yticks(fontsize=9)
plt.show()
del z

- I see that countries like Viet Nam, India, Indonesia, Bangladesh are having high number of young people are interested in field where bigger portion of their participants are younger than 25. 
- Meanwhile countries from Europe, Arabian Peninsula, North East Asia, North America are getting little older in average age distribution. I wouldn't say surely without related data but my inner instincts are telling me that there could be two/three reasons for this:
 * These countries adopted data science education earlier and already have experienced people.
 * People moving abroad to bigger companies clustered in some countries after getting experienced in their home countries.
 * Or some countries having really big population growth as well as interest in data science is rising among the studends.
- Hmm which one it could be...

<img src="https://i.imgur.com/g1NCcmc.png" style="height: 180; width: 100%; object-fit: cover;"/>

# <div class="subsection_detail">2.2.3.2 Gender vs. Country</div>

- While it's important to have diversity among the countries it's also important to have similar diversity between genders too, let's see if we can see different results in that regard...

In [ ]:
# getting percentages and plotting them in heatmap
z=df.groupby([ 'In which country do you currently reside?', 'What is your gender?']).size().unstack().fillna(0).astype('int16')
fig, ax = plt.subplots(figsize=(14, 24))
sns.heatmap(z.apply(lambda x: x/x.sum(), axis=1), xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f', cbar=False)
plt.title('Gender Distribution by Country', fontname = 'monospace', weight='bold')
plt.yticks(fontsize=9)
plt.xticks(fontsize=10)
plt.show()
del z

- Hmm, looks like general trend with global gender distribution is about the same with countries too.
- You can also notice that people who prefer not to share their gender also likely to don't share their location than rest too, I respect their privacy :)

# <div class="subsection_title">2.2.4 Education</div>

- Level of education is a hot topic in data science/machine learning field. Some believe you need a master's degree at least, some claims it's unnecessary. Each on his own I guess, but most people would agree on getting higher degree in field comes with specialized knowledge, career advancements, increased earnings etc, but at the end it's your career choice.
- Now let's listen the story that numbers going to tell us, my hypothesis is either bachelor's or master's going to take first place. You can expect that just by looking at age distribution we checked little while ago...

In [ ]:
count_dist(df=df, colname='What is the highest level of formal education that you have attained or plan to attain within the next 2 years?', max_idx=14, fontsize=20,
           rotation=45, fixlabel=True, f_axis='y', fixlabel_n=3, fixlabel_txt='Some College/Uni Study',
           title='Planned/Current Level of Education of Kaggle Users', palette=cust_color, shift=750, y_label= 'Education')

- Looks like my hypothesis was right (Yay!). Most of the participants are already has master's degree or planning to get one in next two years, that includes me too :)
- By looking at the plot we can see bachelor's and master's are almost at same level, what I wonder is what percentage of master's degree option are actually having that degree or they are just plan to attain. Maybe we can find some insights in future with the features we're going to dig; but for now we can use the age attribute.

# <div class="subsection_detail">2.2.4.1 Education vs Age</div>

- Let's take a closer look to these two degrees with age involved, if we can see something new.

In [ ]:
# Plotting interactive sunburst:

fig = px.sunburst(data_frame=df[(df['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?']=="Master’s degree")|
                               (df['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?']=="Bachelor’s degree")],
                  path=['What is your age (# years)?', 'What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'],
                  color='What is your age (# years)?',
                  color_discrete_sequence=cust_color[::-2],
                  title="Age groups of Master's and Bachelor's degrees"
                 )

fig.update_traces(textinfo='label+percent parent')
fig.show()

- Well it doesn't look like anything to me :) It looks like what it should be looking like, maybe at the age group of 18-21 having some participants master's degree plans, we would assume they are older than 20? Actually it depends on education starting age and education system for specific country so we can't be sure...
- Anyways let's take look at bigger picture with heatmap:

In [ ]:
# getting percentages and plotting them in styled heatmap
z=df.groupby(['What is your age (# years)?','What is the highest level of formal education that you have attained or plan to attain within the next 2 years?']).size().unstack().fillna(0).astype('int16')
fig, ax = plt.subplots(figsize=(14, 12))
sns.heatmap(z.apply(lambda x: x/x.sum(), axis=1), xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f', cbar=False)
plt.title('Planned/Current Education Distribution by Age', fontname = 'monospace', weight='bold')
labels = [item.get_text() for item in ax.get_xticklabels()]
labels[-1] = 'Some College/Uni Study'
ax.set_xticklabels(labels)
plt.xticks(fontsize=12, rotation=60)
plt.yticks(fontsize=12)
plt.xlabel("Education", fontname = 'monospace', weight='semibold')
plt.ylabel("Age", fontname = 'monospace', weight='semibold')
plt.show()
del z

- We can observe that most popular education level starts with bachelor's degree at the age of 18 till 24,
- Then master's degree taking the lead for big chunk of age groups untill age bin of 70+ where doctoral degree taking first place at that point.
- **Special spot for the people who are attending/planning(next 2 years) Doctoral/Professional doctorate degree at the age of 18-21, taking 0.0053% of the age group, you are the real champions :))**

# <div class="subsection_detail">2.2.4.2 Education vs Country</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
z=df.groupby(['In which country do you currently reside?','What is the highest level of formal education that you have attained or plan to attain within the next 2 years?']).size().unstack().fillna(0).astype('int16')
fig, ax = plt.subplots(figsize=(14, 24))
sns.heatmap(z.apply(lambda x: x/x.sum(), axis=1), xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f', cbar=False)
plt.title('Planned/Current Education Distribution by Country', fontname = 'monospace', weight='bold')
labels = [item.get_text() for item in ax.get_xticklabels()]
labels[-1] = 'Some College/Uni Study'
ax.set_xticklabels(labels)
plt.xticks(fontsize=12,rotation=45)
plt.yticks(fontsize=9)
plt.xlabel("Education", fontname = 'monospace', weight='semibold')
plt.ylabel("Country", fontname = 'monospace', weight='semibold')
plt.show()
del z

- Highest master's degree ratio goes to France, which makes sense since 25-29 age group has the biggest percentage in France if you recall from Age chapter and Master's Degree was the most popular one among age 25-29 group globally...
- Highest doctoral rate goes for Denmark, again their age group for country and education looks correlated. Also number of participants from that country should be taken into account.
- In general these Age-Country-Education relation seems solid but still hypothetical, because there are some countries where this trend doesn't fit, like Uganda; this might due to low number of participants.
- Maybe looking at countries where number of participants greater/less than a threshold would give us a cleaner view:

In [ ]:
# plotly treemap for top countries

df['edu_cnt'] = df.groupby('What is the highest level of formal education that you have attained or plan to attain within the next 2 years?')['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'].transform('count')
df['ctr_cnt'] = df.groupby('In which country do you currently reside?')['In which country do you currently reside?'].transform('count')
df['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'].replace({'Some college/university study without earning a bachelor’s degree':'Some Uni Degrees'}, inplace=True)
fig = px.treemap(df[df['ctr_cnt']>df['ctr_cnt'].median()], path=[px.Constant("Countries"),'In which country do you currently reside?',
                                                                 'What is your age (# years)?',
                                                                 'What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'],
                 values='edu_cnt', color='In which country do you currently reside?',
                 color_discrete_sequence=cust_color[::2],
                 title="Country-Age-Education Distribution for Countries With High Number of Participants")
fig.update_traces(textinfo='label+percent parent')
fig.update_layout(margin=dict(t=50, l=0, r=0, b=0))
fig.show()

In [ ]:
# plotly tremap for small countries
fig = px.treemap(df[df['ctr_cnt']<50], path=[px.Constant("Countries"),'In which country do you currently reside?',
                                                                 'What is your age (# years)?',
                                                                 'What is the highest level of formal education that you have attained or plan to attain within the next 2 years?'],
                 values='edu_cnt', color='In which country do you currently reside?',
                 color_discrete_sequence=cust_color[::2],
                 title="Country-Age-Education Distribution for Countries With Low Number of Participants")
fig.update_traces(textinfo='label+percent parent')
fig.update_layout(margin=dict(t=50, l=0, r=0, b=0))
fig.show()

**Note:** You can click on specific tree to see detailed results or hover over them to get better info.

# <div class="subsection_detail">2.2.4.3 What About Courses?</div>

- As we all know the demand for a data scientist is on the rise, at least that's what media says :/
- Students with analytical talents are getting more and more attracted to career in a data science field.
- But usually formal education is not enough or person changing his field without getting another formal education about data science.
- So you will need some specialized/techical education in order to get more data centric roles.
- This brings us to the courses, selecting a right course might boost your career a lot!
- Let's see what course platforms are popular among <span style="color:#00FFFF;"> kaggle </span> users.

In [ ]:
# getting course columns and plotting them
course_cols = [col.split('?')[-1].strip() for col in df.columns if "data science courses" in col]
crs=one_hot_count(df=df, values=course_cols, text="On which platforms have you begun or completed data science courses? ", colors=cust_color, shift_text=1200, x_freq=2000,
                 title="On which platforms have you begun or completed data science courses?", fontsize=16)

- Looks interesting... We do have some leading courses like Coursera or <span style="color:#00FFFF;"> kaggle </span> but still we don't have a clear winner.
- Let's see the effects of courses on other features like formal education.

# <div class="subsection_detail">2.2.4.3.1 Formal Education vs Courses</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(crs.groupby('What is the highest level of formal education that you have attained or plan to attain within the next 2 years?').mean().iloc[:,1:-2],
            xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f', cbar=False)
plt.title('Percentage of Degree Owners who Attended one of the Online Courses', fontname = 'monospace', weight='bold')
plt.ylabel('Education', fontname = 'monospace', weight='semibold')
plt.xlabel('Course Platform', fontname = 'monospace', weight='semibold')
labels = [item.get_text() for item in ax.get_xticklabels()]
labels[-3] = 'Uni Courses'
labels[-4] = 'Cloud Certification'
ax.set_xticklabels(labels)
plt.show()

- Well, we can see the importance of online learning/courses from the table above.
- The higher the person gets formal education there is also higher chance for he/she took one or more courses for data science in past.
- Of course there is time effect on the hypothesis above but still it shows you something for future: Learning never stops in data field!

# <div class="subsection_title">2.2.5 Job / Role</div>

- Ah yes... Another hot topic in data related field: The title... There are really slim differences between some roles or sometimes people are using more inclusionary titles because they do several aspects of the job, like someone doing both data engineering and machine learning. What's his title then? A data engineer? or machine learning engineer? It's hard to answer sometimes, well at least for me :)
- Let's see what we got here in survey, are specialized roles getting their place on top ranks or more generalized titles like data scientist going to take the lead.
- And we shouldn't forget about the age distribution of our data, we're going to have many students...

In [ ]:
count_dist(df=df, colname='Select the title most similar to your current role (or most recent title if retired):',
           max_idx=12, fontsize=16, rotation=45, palette=cust_color,
           title='Role Distribution of Kaggle Users', shift=350, y_label = 'Role'
          )

- Most common role is "Student" by big margin as expected and followed by fellow data scientists...
- By roles alone we can't say much at first look, everything looks what we expected like, we should check relations between this feature and others to see if we can get a better story.

# <div class="subsection_detail">2.2.5.1 Role vs. Age</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
z=df.groupby(['What is your age (# years)?',
              'Select the title most similar to your current role (or most recent title if retired):']).size().unstack().fillna(0).astype('int16')
fig, ax = plt.subplots(figsize=(16, 12))
sns.heatmap(z.apply(lambda x: x/x.sum(), axis=1), xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f', cbar=False)
plt.title('Role Distribution by Age', fontname = 'monospace', weight='bold')
plt.xticks(fontsize=12,rotation=90)
plt.yticks(fontsize=10)
plt.xlabel("Role", fontname = 'monospace', weight='semibold')
plt.ylabel("Age", fontname = 'monospace', weight='semibold')
plt.tight_layout()
del z

- We see that most of the younger people are still studens, who would have expected that right!? Haha...
- Anyways, we can also see that the more participant gets older the more he/she gets a specialized role.
- You can see what I mean above by looking at "Other" role on X axis, percentages almost increasing consistently by age.
- On the other hand we can also observe that data science is still growing field where fraction of data scientists are going down with the age...

# <div class="subsection_detail">2.2.5.2 Role vs. Country</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
z=df.groupby(['In which country do you currently reside?',
              'Select the title most similar to your current role (or most recent title if retired):']).size().unstack().fillna(0).astype('int16')
fig, ax = plt.subplots(figsize=(14, 24))
sns.heatmap(z.apply(lambda x: x/x.sum(), axis=1), xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":10}, fmt='.4f', cbar=False)
plt.title('Role Distribution by Country', fontname = 'monospace', weight='bold')
plt.xticks(fontsize=12,rotation=90)
plt.yticks(fontsize=10)
plt.xlabel("Role", fontname = 'monospace', weight='semibold')
plt.ylabel("Country", fontname = 'monospace', weight='semibold')
plt.show()
del z

- Here we can see roles are mainly occupied by "Student" for most countries. It was expected since most of the participants are students globally.
- Some countries almost having 50% of their participants as students like China and Bangladesh, followed by India and Sri Lanka
- These numbers are cool, but again we don't have balanced distribution for some countries so what I wonder is how are the roles are spread when we count total number of users from these countries.
- This might give us insights about how are the data related roles developing across the world.

# <div class="subsection_detail">2.2.5.3 Role vs. Country Round Two</div>

In [ ]:
# getting std for jobs/roles
x=df.groupby(['In which country do you currently reside?',
              'Select the title most similar to your current role (or most recent title if retired):']).size().unstack().fillna(0).astype('int16')
x =x.apply(lambda x: x/x.sum(), axis=1)
z=x.std(axis=1).sort_values().reset_index().rename(columns={0: 'std'})
df['cnt_count'] = df.groupby('In which country do you currently reside?')['In which country do you currently reside?'].transform('count')
z = z.merge(df[['cnt_count', 'In which country do you currently reside?']], how='left', on='In which country do you currently reside?')
z.drop_duplicates(inplace=True)
df.drop('cnt_count', axis=1, inplace=True)

In [ ]:
# plotting std with counts by countries together
fig, ax = plt.subplots(figsize=(12,8))
sns.barplot(x="In which country do you currently reside?", y="cnt_count", data=z, palette=cust_color, edgecolor='black')
ax2 = ax.twinx()
ax.grid(None)
sns.lineplot(data=z, x="In which country do you currently reside?", y="std", color=cust_color[-1])
fig.autofmt_xdate(rotation=90)
labels = [item.get_text() for item in ax.get_xticklabels()]
labels[30] = 'UK'
ax.set_xticklabels(labels)
ax.xaxis.set_tick_params(labelsize=8)
plt.setp( ax.xaxis.get_majorticklabels(), rotation=-45, ha="left", rotation_mode="anchor") 
plt.title('Standard Deviation of Roles', fontname = 'monospace', weight='bold')
ax.set_ylabel("# of Users", fontname = 'monospace', weight='semibold')
ax2.set_ylabel("Standard Deviation", fontname = 'monospace', weight='semibold')
fig.tight_layout()

- That's more like it! When we include standard deviation of role percentages per country and total participants from that country it gives us somewhat more meaningful story, well at least that's what I thought :/
- So, on X axis we have the countries, left-y axis is showing total number of participants and on right side we have std values for roles of that country.
- On the right side of the X axis we can see the countries where roles are away from normal distribution, this can be caused by couple reasons like:
    - There is a rush/hype for specific role. If demand is not enough for these roles (especially for students entering the field) it might cause some problems in future.
    - Or there isn't many participants to fill every role with healthy distribution, like Norway, we can spot them if they have tiny bars on plot...
- Left side of the chart shows more healthy growth in my opinion, roles are getting saturated after some time, with increased experince etc...

# <div class="subsection_detail">2.2.5.4 Education vs. Role</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
z=df.groupby(['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?',
              'Select the title most similar to your current role (or most recent title if retired):']).size().unstack().fillna(0).astype('int16')
fig, ax = plt.subplots(figsize=(14, 10))
sns.heatmap(z.apply(lambda x: x/x.sum(), axis=1), xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, square=True, cbar=False, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f')
plt.title('Education Distribution by Role')
labels = [item.get_text() for item in ax.get_yticklabels()]
labels[-1] = 'Some College/Uni Study'
ax.set_yticklabels(labels)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel('Role', fontname = 'monospace', weight='semibold')
plt.ylabel('Education', fontname = 'monospace', weight='semibold')
plt.show()
del z

- Hmm this one seems interesting... It looks like to me some students couldn't decide between if they should choose Bachelor's Degree or Some College Study?

# <div class="subsection_detail">2.2.5.5 Courses vs. Role</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(crs.groupby('Select the title most similar to your current role (or most recent title if retired):').mean().iloc[:,1:-2],
            xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f', cbar=False)
plt.title('Percentage of Title Owners who Attended one of the Online Courses', fontname = 'monospace', weight='bold')
plt.ylabel('Education', fontname = 'monospace', weight='semibold')
plt.xlabel('Course Platform', fontname = 'monospace', weight='semibold')
labels = [item.get_text() for item in ax.get_xticklabels()]
labels[-3] = 'Uni Courses'
labels[-4] = 'Cloud Certification'
ax.set_xticklabels(labels)
plt.show()

- Again like education, it doesn't matter what stage you're in your career, you still could use help of online courses.
- We can clearly see decent amount of people who already has specialized titles attended an online course in past or still attending one.

# <div class="section_title">2.3 Programming Languages</div>
- Another important topic for data field, what language a person should learn before he gets into data science. Which one is more practical for you: Do you need to access databases, you want to do more comprehensive statistical analysis or the computing speed is only thing matters to you... Python, SQL, R, C/C++?
- All of these would have have different answers depending on person itself, his background, experience, the task etc.
- In this chapter we going to take closer look to this part of the data science...

# <div class="subsection_title">2.3.1 Experience</div>

- Before getting on with languages themselves let's check how experienced the participants are with coding...

In [ ]:
count_dist(df=df, colname='For how many years have you been writing code and/or programming?',
           max_idx=30, fontsize=24, rotation=45,
           title='Coding Experience', shift=700, y_label='Years of Experience'
          )

- Most of the users has less than three years of coding experience. We could see that coming by looking at amount of students from our previous charts, right?
- But we shouldn't underestimate amount of really experienced coders among participants, it seems they still do present reasonable part.
- Next let's check how experience relates with other features of the users...

# <div class="subsection_detail">2.3.1.1 Role vs. Experience</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
z=df.groupby(['Select the title most similar to your current role (or most recent title if retired):',
             'For how many years have you been writing code and/or programming?']).size().unstack().fillna(0).astype('int16')
fig, ax = plt.subplots(figsize=(16, 14))
sns.heatmap(z.apply(lambda x: x/x.sum(), axis=1), xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f', cbar=False)
plt.title('Role by Coding Experience Years', fontname = 'monospace', weight='bold')
plt.xticks(fontsize=12,rotation=60)
plt.yticks(fontsize=12)
plt.xlabel('Coding Experience Years', fontname = 'monospace', weight='semibold')
plt.ylabel('Role', fontname = 'monospace', weight='semibold')
plt.tight_layout()
del z

- Participants who never written code mostly are consists of business analysts, product managers and some various other roles.
- For under one year experience group, unemployed and students are making a big jump, and some data related roles are somewhat closely distributed like ML Engineer, Data Scientist, Data Engineer etc.
- Next we have 1-3 years of experience, which is the biggest group. 44% of students are in this group as they represent biggest role group among total participants too.
    - 1-3 years looks like a balanced group if we don't include the students. They take around ~20-25% of every single role group. There are some roles at higher end of this average like Unemployed and Data Analysts...
- For the 3-10 years of experience range we can observe the beginning of drastic drops at some groups like Students, Unemployed, I believe that's the part where people start establishing their career path choices.
- 10+ years of experience groups where we can observe lowest levels of unemployment and students.
    - Again we can see that Data Scientist, Machine Learning Engineer, Data Analyst roles are not popular among really experienced participants. This migt be due to:
        - Getting more managerial, advocacy roles with increased experience,
        - Experienced developers who interested in data science but havent changed their career path yet...
- Maybe merging this relation with one more layer would give us better insights:

# <div class="subsection_detail">2.3.1.2 Role, Education and Experience</div>

In [ ]:
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram', colors=None):
    # Source: https://www.kaggle.com/iyadavvaibhav/plotly-sankey-with-filters    
    # Don't forget to go and check his work :)
    colorPalette = colors
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
       
    fig = dict(data=[data], layout=layout)
    
    return fig

In [ ]:
# plotting sankey
df['edu_cnt'] = df.groupby('Select the title most similar to your current role (or most recent title if retired):')['Select the title most similar to your current role (or most recent title if retired):'].transform('count')
sank = genSankey(df,cat_cols=['What is the highest level of formal education that you have attained or plan to attain within the next 2 years?','For how many years have you been writing code and/or programming?','Select the title most similar to your current role (or most recent title if retired):'],value_cols='edu_cnt',title='Education, Experience and Role', colors = cust_color[::4])
fig = go.Figure(sank)
fig.update_layout(margin=dict(t=35, l=20, r=20, b=20))
iplot(fig)

- Ok, looks little bit better,
- We can see the more experience and education the more specialized roles are taken.
- Meanwhile most bachelor's and master's degrees are associated with less than 4 years of experience and mostly students...
- Connection between Master's degree and data scientist role gets little bit stronger with 3-5 years of experience than previous experience groups.


# <div class="subsection_title">2.3.2 The Languages</div>

In [ ]:
languages = ['Python', 'R', 'SQL', 'C', 'C++', 'Java', 'Javascript', 'Julia', 'Swift', 'Bash', 'MATLAB', 'None', 'Other']

In [ ]:
df2=one_hot_count(df=df, values=languages, text='What programming languages do you use on a regular basis? ', colors=cust_color, shift_text=1200, x_freq=2000,
                 title="What's the most popular Programming Language Among Kaggle Users", fontsize=14)

 - Python seems very popular among the <span style="color:#00FFFF;"> kaggle </span> users which is more than twice as much than it's closest competitor, SQL.
 - We can fit C++, R, Java, C, Javascript in same cluster (note that they're not diretly related in other manners of cluster), they just have similar number of users.

# <div class="subsection_detail">2.3.2.1 Role vs. Language</div>
- Here we're going to inspect what languages has been popular with various job titles/roles, so we can get better insights about role demands...

In [ ]:
# getting percentages and plotting them in styled heatmap
fig, ax = plt.subplots(figsize=(16, 14))
sns.heatmap(df2.groupby('Select the title most similar to your current role (or most recent title if retired):').mean().iloc[:,1:-2],
            xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":14}, fmt='.3f', cbar=False)
plt.title('Role and Language Usage', fontname = 'monospace', weight='bold')
plt.ylabel('Role', fontname = 'monospace', weight='semibold')
plt.xlabel('Programming Language', fontname = 'monospace', weight='semibold')
plt.show()

- In general -as we seen previous section- Python is popular with most of the roles. Especially popular among Data Scientists, Machine Learning Engineers and Students.
- Looks like R commonly used by Statisticians,
- SQL has little bit more general usage but, we can see people who work in data pipelines like Database Engineers or Data Engineers it's much more common.
- It seems MATLAB mainly used by Research Scientists.

# <div class="subsection_detail">2.3.2.2 Education vs. Language</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(df2.groupby('What is the highest level of formal education that you have attained or plan to attain within the next 2 years?').mean().iloc[:,1:-2],
            xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f', cbar=False)
plt.title('Education and Language Usage', fontname = 'monospace', weight='bold')
plt.ylabel('Education', fontname = 'monospace', weight='semibold')
plt.xlabel('Programming Language', fontname = 'monospace', weight='semibold')
plt.show()

- We have similar look above, Python is most popular one with all education levels meanwhile SQL takes the second spot,
- It seems R is getting more popular with higher levels of education, is it due to age of the language? Maybe...
- I don't see significant differences for the rest of the programming languages maybe with exception: You can notice people where attended/planning doctorate have used MATLAB previously more common then other groups, this might be due to engineering related education in past...


# <div class="subsection_title">2.3.3 Recommended Language to Learn First</div>

- Let's see what participants are recommending for an aspiring data scientist to learn first.

In [ ]:
count_dist(df=df, colname='What programming language would you recommend an aspiring data scientist to learn first?',
           max_idx=30, fontsize=14, rotation=45,
           title='What Language you Recommend for Beginners', shift=900, y_label='Language'
          )

- Oh wow... Python is the real winner for the beginners. It got recommended almost 14x times more than it's closest follower which is R.
- But I wonder what are the people think after they actually use the language itself. Would they recommend it first again? Or the story changes after getting familiar with that language.


# <div class="subsection_detail">2.3.3.1 Recommending the Language by It's User</div>

Before we start;
- Please don't forget this recommendation is for aspiring Data Scientists and it's recommended for their first choice of programming language.
    - So we are not looking for general purpose language recommendation here,
    - Nor recommendations for experienced data scientists directly.
    - So for example, a person who's using C++ and not recommending that actually not recommending his own language for the beginner data scientists, not because he doesn't like it :)
    - Also people who are using C++ are likely been using other languages like Python too, since it's multi choice question...
- Well, with leaving the warnings behind let's take a look at the data itself :D

In [ ]:
# getting ratios as dict and plot them

ratios = {}

for lan in languages:
    ratio = df2[df2[lan]==1]['What programming language would you recommend an aspiring data scientist to learn first?'].value_counts(normalize=True)
    ratios[lan]=ratio[lan]
recommendations=pd.DataFrame.from_dict(ratios, orient='index').reset_index().rename(columns={'index':'Used Language', 0:'Recommendation'}).sort_values(by='Recommendation', ascending=False)
ax=sns.barplot(y=recommendations['Used Language'], x=recommendations['Recommendation'], palette=cust_color, edgecolor='black', saturation = 1.5, linewidth= 1.5)
plt.ylabel('Language', fontname = 'monospace', weight='semibold')
plt.xlabel('Recommendation Ratio', fontname = 'monospace', weight='semibold')
plt.title('Recommendation Ratios of Languages Where Participant Uses it on a Regular Basis', fontname = 'monospace', weight='bold')
plt.xticks(np.arange(0, 1, 0.05))
plt.show()

- Hmm 84% of the participants who uses Python regularly recommends Python for aspiring data scientist to learn first.
- That makes Python is the most recommended language by it's regular users by far.
- Closest one to this one is R, which is around 20%
- Julia also an interesting one, I think high performance makes it recommendable by it's users. Maybe with some more library support we might see increased numbers incoming years?
- We can already see Python is popular recommendation with all programming language users, so I just want to open new window for Python itself:
    - Who doesn't recommend Python?
    - What other languages are recommended by people who uses Python already?

In [ ]:
# getting ratios as dict and plot them
ratios = {}

for lan in languages:
    ratio = df2[df2[lan]==1]['What programming language would you recommend an aspiring data scientist to learn first?'].value_counts(normalize=True)
    ratios[lan]=ratio['Python']
recommendations=pd.DataFrame.from_dict(ratios, orient='index').reset_index().rename(columns={'index':'Used Language', 0:'Recommendation'}).sort_values(by='Recommendation', ascending=False)[1:]
ax=sns.barplot(y=recommendations['Used Language'], x=recommendations['Recommendation'], palette=cust_color, edgecolor='black', saturation = 1.5, linewidth= 1.5)
plt.ylabel('Language', fontname = 'monospace', weight='semibold')
plt.xlabel('Recommendation Ratio', fontname = 'monospace', weight='semibold')
plt.title('Python Recommendation Ratios', fontname = 'monospace', weight='bold')
plt.xticks(np.arange(0, 1, 0.05))
plt.show()

- It looks like love for Julia and R strong against Python.
- People who uses these two languages I mentioned above more likely to recommend you another beginner data science language than Python :)

In [ ]:
# lineplot for python users
fig, ax = plt.subplots(figsize=(14, 6))
plt.plot(df2[df2['Python']==1]['What programming language would you recommend an aspiring data scientist to learn first?'].value_counts(normalize=True)[1:], color=cust_color[-1],
        linestyle='dashed', marker='o', markerfacecolor='red', markersize=16)
plt.xlabel('Language', fontname = 'monospace', weight='semibold')
plt.ylabel('Recommendation Ratio', fontname = 'monospace', weight='semibold')
plt.title("Recommendations From Users Who Uses Python Regularly (Except Python)", fontname = 'monospace', weight='bold')
plt.yticks(np.arange(0, 0.05, 0.0025))
plt.show()

- It's really small part of the users recommend other languages in this group (remember they mostly recommended Python), people who uses Python but doesn't recommend python are tend to recommend SQL (around %5) and R (around 5%) for aspiring data scientists.

# <div class="section_title">2.4 Work Platforms</div>

- In this chapter we're going see some stuff like development environments (IDE's), hosted notebook products, computing platforms, specialized hardwares, tensor processing units (TPU's) etc. Ok maybe not all of them but still :)
- Some of them might not sound familiar to you, but don't worry, I'm going to give short describe about most these terms quite soon.
- But basically in this chapter we're going to inspect which platforms (both software and hardware) participants using their daily data science / machine learning projects.
- Well, let's get the ball rolling then!

# <div class="subsection_title">2.4.1 IDE's</div>

- Here we're going to see some of the tools can make your life easier while coding.
- Each has it's own pro's and con's, usually there is no agreement among people about which one is best.
- In my opinion you should look at these as tools yourself and choose the best fitting one.

In [ ]:
# setting multichoice ide columns
ide_cols = [col.split('?')[-1].strip() for col in df.columns if "IDE's" in col]

In [ ]:
df3=one_hot_count(df=df, values=ide_cols, text="Which of the following integrated development environments (IDE's) do you use on a regular basis? ", colors=cust_color, shift_text=1200, x_freq=2000,
                 title="Which IDE's do you use on a regular basis?", fontsize=14)

- Looks like Jupyter products are the most popular one, they are easy to use, good for exploration and prototyping.
- Then comes VSCode and PyCharm, these are more for writing a production code, where you deploy your code to actual use or something like that.
- There might be some relations like "RStudio probably more popular among the R users", (thanks to the Captain Obvious), but let's see if there's an actual connection between languages and IDE's:

# <div class="subsection_detail">2.4.1.1 IDE vs Language</div>

- What we going to find about in next plot is whetever if there is a correlation with IDE and user's programming language preference:

In [ ]:
# creating masked heatmap for correlations
df4 = pd.concat([df3, df2[languages].add_suffix('_language')], axis=1)
correlation_train = df4[ide_cols+[lang+'_language'for lang in languages]].corr(method='kendall')
mask = np.triu(correlation_train.corr())
plt.figure(figsize=(16, 16))
sns.heatmap(correlation_train,
            annot=True,
            fmt='.2f',
            cmap='YlOrBr',
            square=True,
            mask=mask,
            
            cbar=False,
            annot_kws={"fontsize":10})

plt.xlabel('')
plt.ylabel('')
plt.title("IDE / Language Usage", fontname = 'monospace', weight='bold')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.show()

- Interesting... Even sometimes it's hard to find meaningful correlations between categorical datas we can see some noticable correlations between some categories.
- The most obvious one is between R and RStudio,
- With MATLAB and MATLAB IDE there we can notice similar correlation
- There's noticeable correlation between Python and Jupyter Notebooks
- Vim / Emacs has some degree of correlation with Bash 
- While looking at this table we can also see some kind of clusters between some languages like: Java, Javascript, C, C++, it shows when a participant uses one of these languages regularly there is a stronger chance than rest to use one of the other languages from this cluster...

# <div class="subsection_title">2.4.2 Notebooks</div>

In [ ]:
# setting multi choice notebook column names as list
notebook_cols = [col.split('?')[-1].strip() for col in df.columns if "notebook products" in col]

df5=one_hot_count(df=df, values=notebook_cols, text="Which of the following hosted notebook products do you use on a regular basis? ", colors=cust_color, shift_text=1000, x_freq=2000,
                 title="Which Notebooks do you use on a regular basis?", fontsize=12)

- Colab and <span style="color:#00FFFF;"> kaggle </span> notebooks are the leaders here. 
- We also have significant amount of people who doesn't prefer notebooks at all too.
- It comes back to topic we discussed before, it's important to decide what tool you going to use depending on your project before you start it...

# <div class="top_section">It Was All Connected! Was it?..</div>


<img src="https://i.kym-cdn.com/entries/icons/original/000/022/524/tumblr_o16n2kBlpX1ta3qyvo1_1280.jpg" style="height: 180; width: 100%; object-fit: cover;"/>

[Source: Know Your Meme](https://knowyourmeme.com/memes/pepe-silvia)

- So, in previous chapters we went over somewhat more general attributes of a <span style="color:#00FFFF;"> kaggle </span> user. I went with a hypothesis that almost all participants are somewhat interested with programming and data itself to some degree. As we seen, it's almost true...

- We have seen some strong, some weak connections but at the end we found some connections between most of the variables.
- But what are we after here? Where all these connections lead to?
- I mean we can find direct connections between two or more features but what do they mean in general picture?
- I believe we can find some key features that many of subfeatures might lead to, like:
    - Role
    - Salary
    - Education
- Or maybe something else, we don't know yet but that's what we are trying to find.
- I'm hoping we going to find answers for these questions in next chapters which hopefully our story gonna unfold with the help of data. Were they all connected? I don't know, but I hope so. Let's get going to find it!
- But before doing that, let's talk about something else too:

# <div class="section_title">Story of Robert</div>

- So we inspected and searched for relationships between demographics, background, some general programming knowledge and preference of working environment.
- While doing so we learned a lot about an average <span style="color:#00FFFF;"> kaggle </span> user:
    - Demographics of a participant like, age, gender, place of residence.
    - Educational background, what's the highest level of education or plans for next two years, courses attended etc.
    - Actual role of a participant, we learned about if a participant is currently unemployed, studying or working, if so what's the title.
    - Coding related information like, coding experience in years, programming language usage, recommendations for beginners etc.
    - Working environment info, which IDE a participant uses regularly, choice of notebook platform of a given <span style="color:#00FFFF;"> kaggle </span> user.
    
With all of these info I believe we can create an "Average Joe" of <span style="color:#00FFFF;"> kaggle </span> if we randomly select some of the popular features. Well let's create one then! We can call him "Robert", why not?
### **The Story of our little Robert would be something like this:**

*Robert is a 24 year old male from USA, he got his Bachelor's degree recently and planning to attend Master's program incoming years. He still identifies himself as a student. He's been interested in programming and data science lately but didn't have formal education to get to know about these in school. So he decided to learn programming from online courses on his own. He found out Python is very popular for Data Science and he started studying about it. After couple months of courses (1< years of experience) he feels competent enough to look at Data Science stuff using Python. He jumps on <span style="color:#00FFFF;"> kaggle </span> to play with toy datasets like Titanic, Housing Prices etc. He gets more familiar to work on <span style="color:#00FFFF;"> kaggle </span> (jupyter) notebooks but with all of the new stuff about data science pouring over him like Data Analysis, Machine Learning, Deep Learning Frameworks, Algorithms, CV/NLP, he gets little bit overwhelmed and lost... He doesn't know what to study and in which order he should start learning. Even he learns about some stuff, he still gets confused about his future job, is he going to be a Data Analyst? Data Scientist? Or Machine Learning Engineer? Or something else... He doesn't know, they all seem similar to him and with his background/skills he doesn't know which role he perfectly fits. What is going to be his daily work responsibilities, what does he need to know to fulfill his duties, what does he need to learn to advance in his career. Will his salary going to be satisfying for him, or did he make a mistake by choosing Data Science field? What if he made a bad decision, or he's going to make one? Robert gets stressed and confused lately... Maybe with help of the data itself we can help poor little Robert, shall we?*


<img src='https://image.freepik.com/free-vector/worried-concept-illustration_114360-3729.jpg' width=600>
<div align="center"><font size="1">Source: <a href="https://www.freepik.com/free-vector/worried-concept-illustration_10386551.htm">Worried by "Stories" - www.freepik.com</a></font></div>  

# <div class="section_title">2.5 Time to Specialize</div>

- In this chapter we're going take a closer look to the needs of data related roles, what libraries, frameworks, algorithms etc. you need to learn to seperate you from others who already can code.
- What makes you a data specialist? What are the tools people have been using for this purpose? What are the tools which has more general use while which ones are for more specific/specialized roles.

*It's going to be really useful chapter for Robert, he's been really confused with what to learn next; and learning about what tools people are using regularly might help him. If we can connect these with specific career path it's going to be even more helpful! Lucky you Robert, things are already getting better for you! :)*

# <div class="subsection_title">2.5.1 Visualization Libraries</div>

In [ ]:
# setting multi choice visualization column names as list
visualization_cols = [col.split('?')[-1].strip() for col in df.columns if "visualization" in col]
df6=one_hot_count(df=df, values=visualization_cols, text="What data visualization libraries or tools do you use on a regular basis? ", colors=cust_color, shift_text=1200, x_freq=2000,
                 title="What data visualization libraries or tools do you use on a regular basis?", fontsize=14)

# <div class="subsection_detail">2.5.1.1 Visualization Libraries and Languages</div>

In [ ]:
# creating masked heatmap for correlations
df6 = pd.concat([df6, df2[languages].add_suffix('_language')], axis=1)
correlation_train = df6[visualization_cols+[lang+'_language'for lang in languages]].corr(method='kendall')
mask = np.triu(correlation_train.corr())
plt.figure(figsize=(16, 16))
sns.heatmap(correlation_train,
            annot=True,
            fmt='.2f',
            cmap='YlOrBr',
            square=True,
            mask=mask,
            
            cbar=False,
            annot_kws={"fontsize":10})

plt.xlabel('')
plt.ylabel('')
plt.title("Visualization Library / Language Correlations", fontname = 'monospace', weight='bold')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.show()

- I see some decent correlations here, most obvious one is Ggplot / ggplot2 and R correlation, well it was expected, right? I know some people are using R just for ggplot...
- Seaborn & Matplotlib are closely related. Well... One built over the other one anyways...
- With Python most correlated ones are these two I mentioned above, followed by Plotly which gets pretty popular lately.

*For Robert we can suggest taking a closer look to Seaborn & Matplotlib, since he feels competent with Python. We still don't know if he is going to like data visualization though. But anyways, knowing little bit of visualization might help a lot in his other tasks too. I can also recommend Plotly to Robert if he's interested in visualization more, lately I'm seeing lots of cool plots created with it around here :)*

# <div class="subsection_detail">2.5.1.2 Visualization Libraries and Roles</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(df6.groupby('Select the title most similar to your current role (or most recent title if retired):')[visualization_cols].mean(),
            xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f', cbar=False)
plt.title('Percentage of Participants Who Uses a Certain Visualization Library', fontname = 'monospace', weight='bold')
plt.ylabel('Job / Role', fontname = 'monospace', weight='semibold')
plt.xlabel('Visualization Library', fontname = 'monospace', weight='semibold')
plt.xticks(rotation=60, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.show()

- Doesn't matter your job/role is you probably used Matplotlib once or twice. The table above shows about 60% of the participants uses that library regularly.
- Statisticians are using ggplot more often, we can observe the Statistician, R and ggplot relation here easily.
- Machine Learning Engineers who uses Matplotlib regularly has bigger percentage than Data Scientists. I believe it's because of matplotlib's easy to use and it's way get results in a fast and simple fashion, meanwhile seaborn and plotly a little bit more popular for Data Scientists; probably for the people who needs to present their findings in better shape :)

*Well Robert, I know you are confused and still not sure what data oriented career path you should choose in future but you must know about at least one visualization library. As you can see it doesn't matter if you going to be Data Analyst or Machine Learning Engineer, you need to visualize your findings and use/present them in your job. We already suggested which ones to start learning first but these findings are strengthening the idea.*

# <div class="subsection_title">2.5.2 Machine Learning Frameworks</div>

In [ ]:
# setting multi choice machine learning column names as list and plotting them
ml_cols = [col.split('?')[-1].strip() for col in df.columns if "machine learning frameworks" in col]
df7=one_hot_count(df=df, values=ml_cols, text="Which of the following machine learning frameworks do you use on a regular basis? ", colors=cust_color, shift_text=1000, x_freq=2000,
                 title="Which of the following machine learning frameworks do you use on a regular basis?", fontsize=12)

- Things are getting more complicated here :)
- With ML libraries/frameworks are entering the picture, even the clear winner barely passes the 50% limit, which is Scikit-learn.
- Since I use most of these on regular basis I might add some personal notes about them:
    - Scikit-learn is a library provides many learning algorithms, setting standard for many other ML libraries. It's a great tool for prototyping, I forgot how many times I started prototyping a solution for a problem and at the end found out sklearn is more than enough...
    - Tensorflow is a library for fast numerical computings which we need them for Deep Learning. It's has low level abilities to customize your workflow as you want.
    - Keras is higher level API for Tensorflow, it wraps most common deep learning tasks aiming for easier usage. (You can still do low level stuff though using funtional or subclassing API's)
    - Pytorch is somewhere between these two above, again it's an open source ML library for mainly deep learning tasks. People usually say it has more "Pythonic" style...
     - You can think of PyTorch Lightning like a Keras for Pytorch, not exactly same but it's aiming for similar purposes.
    - JAX is relatively a young framework, it's designed for high-performance numerical computing. People like to call it "NumPy on steroids" :) I find that description pretty accurate too :)
    - Xgboost, LightGBM and Catboost are similar libraries, aiming to get maximum use of gradient boosting. They do well on tabular data usually...
    - You can also see some libraries has more specific usage areas like Huggingface for Natural Language Processing tasks or Prophet for time series etc.

# <div class="subsection_detail">2.5.2.1 Machine Learning Libraries and Roles</div>

In [ ]:
# getting percentages and plotting them in styled heatmap

fig, ax = plt.subplots(figsize=(16, 8))
sns.heatmap(df7.groupby('Select the title most similar to your current role (or most recent title if retired):')[ml_cols].mean(),
            xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":10}, fmt='.4f', cbar=False)
plt.title('Percentage of Participants Who Uses a Certain ML Library', fontname = 'monospace', weight='bold')
plt.ylabel('Job / Role', fontname = 'monospace', weight='semibold')
plt.xlabel('ML Library', fontname = 'monospace', weight='semibold')
plt.xticks(rotation=60, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.show()

- Scikit-learn seems to have wide variety users, doesn't matter what role you have it's still has some use cases for you. It has pretty common usage with Data Scientists and Machine Learning Engineers.
- When we switch to Deep Learning Frameworks like TensorFlow, Keras, PyTorch, we see Machine Learning Engineers are getting one step ahead, while most roles seen decrease from sklearn to these frameworks, one of the most limited decrease is from ML Engineers. Followed by Research Scientists and Sofware Engineers (Interesting one)...
- Gradient boosting algorithms are more common than other role groups among Data Scientists.
- It seems statisticians are one of the least interested groups in machine learning (after Developer Relations), I was expecting little big jump at Tidymodels for this group but it seems even "None" group had sharper spike.

*So Robert... I heard you getting interested specially in Data Scientist and Machine Learning Engineer roles. You better start learning Scikit-learn then, as you can see it's really popular and in demand. Also, like I mentioned above it has great workflow standard, so after getting familiar with it you can learn other libraries much faster. For example some popular libraries like Xgboost, LightGBM already having sklearn interface too, so after getting familiar with sklearn you can start using these too in no time! Good lad! If you want to advance further I'd suggest taking a look at the TensorFlow and especially Keras, because of it's higher level API and having some similarities with sklearn workflow, it would be good starting point. If I recall correctly Keras has sklearn wrapper too. To get little bit of theory and practical knowledge you can get help of some books. For Scikit-learn I'd recommend "Introduction to Machine Learning with Python: A Guide for Data Scientists" from Andreas C. Müller, for TensorFlow and Keras you can take a look at "Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow" from Aurélien Géron, in that book you can get some solid knowledge for sklearn too. If you want something Keras specific you can read "Deep Learning with Python" from François Chollet. Getting familiar with these going to take some time but after that you can start learning PyTorch if you like to, it's a great framework and lately on the rise, many research codes are published in PyTorch format so it's nice to know. Hey Robert! Are you still there?!*

# <div class="subsection_detail">2.5.2.2 Machine Learning Libraries and Visualization</div>
- So, couple sections above we have seen visualization libraries are pretty popular among Machine Learning practicioners, let's see how these relations connects with ML frameworks themselves.

In [ ]:
# plotting correlations as regplot

df7 = pd.concat([df7, df6[visualization_cols]], axis=1)
z= df7[visualization_cols+ml_cols]
corr_mat = z.corr().stack()
z=pd.DataFrame(data=corr_mat.values, index=corr_mat.index.to_flat_index(), columns=['Correlation']).reset_index()
z['index'] = z['index'].astype('str')
z['index'] = z['index'].str.replace('(','', regex=False)
z['index'] = z['index'].str.replace(')','', regex=False)
z[['level_0','level_1']] = z['index'].str.split(', ',expand=True)

g = sns.relplot(
    data=z,
    x="level_0", y="level_1", hue="Correlation", size="Correlation",
    palette="YlOrBr", hue_norm=(-1, 1), edgecolor=".9",
    height=10, sizes=(50, 250), size_norm=(-.2, .8),
)

g.set(xlabel="", ylabel="", aspect="equal")
g.ax.margins(.02)


for label in g.ax.get_xticklabels():
    label.set_rotation(90)
for artist in g.legend.legendHandles:
    artist.set_edgecolor(".7")
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.title('Machine Learning Framework and Visualization Library Correlations', fontname = 'monospace', weight='bold')
plt.show()

- Most popular visualization tools (Matplotlib, Seaborn) are highly related with Scikit-learn, followed by Keras/TF and XGboost.
- ggplot and Shiny are more correlated with Tidymodels and Caret. Again strong R connections :)
- For the rest of visualization libraries I couldn't find meaningful correlations between them and ML libraries, so let's check relations between ML libraries themselves next.
- You can notice decent correlation between sklearn and gradient boosting algorithms and TF/Keras, so in some ways that confirms our suggestion to learn Scikit-learn first.
- For PyTorch strongest ones are Pytorch Lightning (Surprise!) and Huggingface, I think you can use Tensorflow with Huggingface too but PyTorch seems more default setting for it. Again TF/Keras has some correlation with PyTorch too since they both serve for Deep Learning...

*You see Robert? Now we have stronger foundations for the suggestions about your data science roadmap. As you can see the higher correlated libraries are creating a cluster for most of the data scientists: Matplotlib/Seaborn usage correlated with Scikit-learn, Scikitlearn usage correlated with gradient boosting algorithms and these are somewhat correlated with deep learning frameworks like TF/Keras. With these small steps we start getting a broad roadmap for what you need to learn already, how you feel about that?*

# <div class="subsection_title">2.5.3 Machine Learning Algorithms</div>

In [ ]:
# setting multi choice algorithm column names as list and plotting them
alg_cols = [col.split('?')[-1].strip() for col in df.columns if "ML algorithms" in col]
df8=one_hot_count(df=df, values=alg_cols, text="Which of the following ML algorithms do you use on a regular basis? ", colors=cust_color, shift_text=1200, x_freq=2000,
                 title="Which of the following ML algorithms do you use on a regular basis?", fontsize=16)

- Regression tasks (either linear or logistic for classification) are most popular ML algorithms by participants.
- Tree based algorithms taking second place, again pretty common among <span style="color:#00FFFF;"> kaggle </span> users.
- Gradient boosting and CNN's (mostly for computer vision tasks) following these two.
- It's interesting to see DNN's are less popular than CNN's, probably classical supervised approaches (sklearn and gradient boosting stuff) are still preferable for tabular data...
- Then comes Bayesian Approaches, RNN's and Transformers... Last two are usually used for Natural Language Processing problems but lately Transformers are getting some hype around computer vision tasks too.

# <div class="subsection_detail">2.5.3.1 Machine Learning Algorithms and Roles</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
fig, ax = plt.subplots(figsize=(16, 10))
sns.heatmap(df8.groupby('Select the title most similar to your current role (or most recent title if retired):')[alg_cols].mean(),
            xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":14}, fmt='.4f', cbar=False)
plt.title('Percentage of Participants Who Uses a Certain Algorithm', fontname = 'monospace', weight='bold')
plt.ylabel('Job / Role', fontname = 'monospace', weight='semibold')
plt.xlabel('Algorithm', fontname = 'monospace', weight='semibold')
plt.xticks(rotation=60, ha='right', fontsize=10)
plt.yticks(fontsize=10)
labels = [item.get_text() for item in ax.get_xticklabels()]
labels[2] = 'Gradient Boosting'
ax.set_xticklabels(labels)
plt.show()

- The heatmap above you can see which role regularly interacts with a certain algorithm. 
- Again we can confirm that Data Scientist is a broad term, this role uses wide range of algorithms regularly.
- Machine Learning Engineer role uses neural networks more than rest, this includes:
    - DNN's
    - CNN's
    - RNN's
    - Transformers
    - GAN's
- These above are more specialized algorithms for tasks like computer vision, natural language processing, autoencoders etc.
- Research scientists also having similar algorithm usage patterns with machine learning engineers.

*Hey there Robert! You probably heard enough but I'm going to say again, learn Scikit-learn! You can use most of these algorithms within the sklearn package, even some DNN's like Perceptron or MLP's and as you can see you are going to use some of these algorithms a lot and it doesn't matter what specific title you want to get in future. You were interested in Data Scientist or Machine Learning Engineer roles right? I'd definitely start learning then... I can show it to you in another way, look:*

In [ ]:
# plotting sankey diagram using px
df7['Using_Matplotlib_or_Seaborn'] = np.where((df7['Matplotlib'] == 1) | 
                           (df7['Seaborn'] == 1),
                                                    'Yes', 'No')

sankey_df = df7[df7['Select the title most similar to your current role (or most recent title if retired):']!='Student']
sankey_df['Using_Scikit-learn']= sankey_df['Scikit-learn'].map({0:'No', 1:'Yes'})
sankey_df['title_count'] = sankey_df.groupby('Select the title most similar to your current role (or most recent title if retired):')['Select the title most similar to your current role (or most recent title if retired):'].transform('count')
fig = px.parallel_categories(sankey_df,dimensions=['Using_Matplotlib_or_Seaborn','Using_Scikit-learn',
                                           'Select the title most similar to your current role (or most recent title if retired):'],
                              labels={'For how many years have you used machine learning methods?':'Machine Learning Experience',
                                          'Select the title most similar to your current role (or most recent title if retired):':'Job/Role'},
                             color='title_count',color_continuous_scale=cust_color[::-1])

fig.update_layout(
    hovermode = 'x',
    title="<b>Matplotlib/Seaborn > Scikit-learn path to Roles</b>",
    font=dict(size = 14, color = 'black'),
    titlefont=dict(size =16)
)
fig.update_coloraxes(showscale=False)
fig.update_layout(margin=dict(t=40, l=70, r=140, b=20))
fig.show()

As you can see there is a strong path to the Data Scientist and Machine Learning Engineer roles from classical libraries.

# <div class="subsection_detail">2.5.3.2 Machine Learning Algorithms and Libraries</div>

In [ ]:
# plotting correlations as regplot
df8 = pd.concat([df8, df7[[col for col in ml_cols if col not in ['Other', 'None']]]], axis=1)
z= df8[alg_cols+ml_cols]
corr_mat = z.corr().stack()
z=pd.DataFrame(data=corr_mat.values, index=corr_mat.index.to_flat_index(), columns=['Correlation']).reset_index()
z['index'] = z['index'].astype('str')
z['index'] = z['index'].str.replace('(','', regex=False)
z['index'] = z['index'].str.replace(')','', regex=False)
z[['level_0','level_1']] = z['index'].str.split('\',',expand=True)

g = sns.relplot(
    data=z,
    x="level_0", y="level_1", hue="Correlation", size="Correlation",
    palette="YlOrBr", hue_norm=(-1, 1), edgecolor=".9",
    height=15, sizes=(50, 250), size_norm=(-.2, .8),
)

g.set(xlabel="", ylabel="", aspect="equal")
g.ax.margins(.02)
for label in g.ax.get_xticklabels():
    label.set_rotation(90)
for artist in g.legend.legendHandles:
    artist.set_edgecolor(".7")
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.title('Machine Learning Library and Alghorithm Correlations', fontname = 'monospace', weight='bold')
plt.show()

- The plot above confirms that Scikit-learn is correlated with most of the machine learning algorithms.
- For neural networks TF/Keras, PyTorch is getting more popular.
- And lastly as expected from previous plots Transformer Networks are highly correlated with Huggingface

# <div class="subsection_title">2.5.4 Machine Learning Algorithms</div>

# <div class="subsection_detail">2.5.4.1 ML Experience Distribution by Role</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
z=df.groupby(['Select the title most similar to your current role (or most recent title if retired):',
    'For how many years have you used machine learning methods?',
              ]).size().unstack().fillna(0).astype('int16')
fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(z.apply(lambda x: x/x.sum(), axis=1), xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":10}, fmt='.4f', cbar=False)
plt.title('ML Experience Distribution by Role', fontname = 'monospace', weight='bold')
plt.xticks(fontsize=12,rotation=90)
plt.yticks(fontsize=12)
plt.ylabel("Role", fontname = 'monospace', weight='semibold')
plt.xlabel("Experience in Machine Learning (in Years)", fontname = 'monospace', weight='semibold')
plt.show()
del z

*You see Robert? No need to worry about getting confused with data science concepts at first sight, as you can see most of the <span style="color:#00FFFF;"> kaggle </span> users are either under 1 years of ML experience or has no experience at all. It's still young field and there will be many career opportunities if you play your cards right and in time!*

# <div class="subsection_detail">2.5.4.2 ML and Programmin Experience</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
z=df.groupby(['For how many years have you used machine learning methods?',
    'For how many years have you been writing code and/or programming?',
    
              ]).size().unstack().fillna(0).astype('int16')
fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(z.apply(lambda x: x/x.sum(), axis=1), xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f', cbar=False)
plt.title('Coding Experience Distribution by ML Experience', fontname = 'monospace', weight='bold')
plt.xticks(fontsize=12,rotation=90)
plt.yticks(fontsize=12)
plt.ylabel("Years of ML Experience", fontname = 'monospace', weight='semibold')
plt.xlabel("Years of Coding Experience", fontname = 'monospace', weight='semibold')
plt.show()
del z

- We can see solid linear correlation between years of programming experience and years of ML experience.
- Usually ML experience of participants are couple years less than their programming experience, which makes sense...
- I noticed there are **6 people with more than 20 years of Machine Learning experience but less than 1 years of coding experience**. Well, I mean it's important to learn theory behind the ML algorithms before start coding them, right? Haha...

*It's a good thing you start learning how to code with Python before you start doing Machine Learning stuff. As I mentioned above usually people are getting familiar with a programming language before they start learning the algorithms etc. You done well there Robert, good lad!*

# <div class="section_title">2.6 The Industry</div>

- So, we learned a lot about the skills themselves in previous chapter. Now we have general idea about what tools you're going to need in your tool belt before entering into industry. Of course you could learn most of these while working or improve what you've learned before while working too, the learning never stops! But it's a good thing to know some concepts about the field before you dive directly into it...
- Now in this chapter we're going to learn where and how to use these skills in industry.
- Probably every role has different responsibilities and different needs, so it's nice to know about them before you decide which areas you should specialize or where to apply these skills.
- It's important to know how is the future of data related jobs in industry before you spending time on learning their needs.
- Maybe you'd like to know about how much money you going to make in next years before you take important decisions in your life...

*Alrighty Robert! You still studying right? Well of course you do... Anyways it's time to take a look at the field itself before you start applying jobs, maybe you can fine grain your options and improve some more specific skills after learning about these :) Like which area you going to work, what's your responsibilities going to be at work, what skills you need for these responsibilities and **ahem** of course how much you are going to earn, and a lot more! Your time is coming Robert, exciting!*

# <div class="subsection_title">2.6.1 Work Activities</div>

- Let's stary by looking what's your daily responsibilities going to be like in future job...

In [ ]:
activities = [col.split(':')[-1].strip() for col in df.columns if "an important part" in col]
df10=one_hot_count(df=df, values=activities, text="Select any activities that make up an important part of your role at work: ", colors=cust_color, shift_text=1500, x_freq=2000,
                 title="Activities that make up an important part of your role at work", fontsize=20)

- Seems like analyzing and understanding the data for stakeholders is still most common task for data related jobs.
- Second most popular activity seems intriguing, looks like industry is still trying to find use of machine learning techniques to take advantage of. So I believe there will be new areas to explore and conquer :) 
- If we merge these two above we can see that we're still in early parts of machine learning part of the data science, lot to discover, lot to understand...
- Third one more relevant to data itself, creating and storing useful information is really important.
- Next two seems like using ML techniques more practically, where participant tries to build/upgrade machine learning methods to improve his workflow.
- Doing research probably important for research scientists or maybe machine learning engineers, we'll find about it soon.
- And lastly there is decent portion of participants who doesn't find these activities important for his role...

# <div class="subsection_detail">2.6.1.1 Roles and Responsibilities</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
fig, ax = plt.subplots(figsize=(16, 10))
sns.heatmap(df10.groupby('Select the title most similar to your current role (or most recent title if retired):')[activities].mean(),
            xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f', cbar=False)
plt.title('Percentage of Work Activities by Role', fontname = 'monospace', weight='bold')
plt.ylabel('Job / Role', fontname = 'monospace', weight='semibold')
plt.xlabel('Activity', fontname = 'monospace', weight='semibold')
plt.xticks(rotation=60, ha='right', fontsize=10)
plt.yticks(fontsize=10)

labels = [item.get_text() for item in ax.get_xticklabels()]
labels = ['Analyze & Understand Data', ' Build & Run Data Infrastructure', ' Build ML Prototypes for New Areas', 'Build/Run ML Services', 'Experiment/Improve Existing ML Models',
         'Researching Advances in ML', 'None of these are Important', 'Other']
ax.set_xticklabels(labels)

plt.show()

- This one above seems interesting... We can learn a lot about roles and their responsibilities.
- Analyzing and understanding the data is pretty important for all roles, maybe with the exceptions like other roles or software engineers...
    - For Business and Data analysts learning the data and influence business decisions are most important thing. (Well, surprise...)
    - Again for data scientists exploring the data seems pretty important but as you can see it's not the only responsibility they usually take :)
    - We can also see that understanding the data is important for managerial roles too...
- Again an obvious one, building and running data infrastructure is pretty common for data and database engineers.
- Building machine learning prototypes for new areas to exploit seems pretty common among the data&research scientists and ML engineers. An interesting one for me is  the product manager one which more than one third of product managers are prototyping machine learning to new areas.
- Building and running ML services are less common than I expected, mostly taken by machine learning engineers and data scientists.
- Experimenting or improving models taking important part of ml engineers and data&research scientists daily duties.
- For research, research scientists are coming first (Duh!) followed by ml engineers. 

*Well Robert, whatever you do for work, to find a place in data field you must understand and process the data. As I told you in previous chapter best way to do this is doing exploratory data analysis and one of the best tools going to help you with this is visualization. That's why it's recommended to learn matplotlib/seaborn for you. When you understand the data you can move to the next steps like asking the questions: "What can I do with this data, how can I use this information to create something useful, can I train a model to predict stuff with this data". If you start asking these kind of questions you're on the right track, but again you need to get hang of the data first, then the questions are getting more specialized. After exploring, to answer some of these questions you going to use machine learning. It's going to take lot's of prototyping and as I mentioned in previous chapter, Sckit-learn going to help you a lot with this. You don't want to create complex models before you reach satisfactory results with more traditional models. If you manage to do this then you'll get to next step where you need to improve existing model, maybe doing some research to learn about state of the art stuff then deploy the model etc. Maybe you are little bit unexperienced for some of these but it's good to know and by the way please stop asking me about the wage, we'll get there...*

# <div class="subsection_detail">2.6.1.2 ML Experience and Responsibilities</div>

In [ ]:
# getting percentages and plotting them in styled heatmap
fig, ax = plt.subplots(figsize=(16, 10))
sns.heatmap(df10.groupby('For how many years have you used machine learning methods?')[activities].mean(),
            xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":14}, fmt='.4f', cbar=False)
plt.title('Percentage of Work Activities by Experience', fontname = 'monospace', weight='bold')
plt.ylabel('Experience in ML (# years)', fontname = 'monospace', weight='semibold')
plt.xlabel('Work Responsibilities', fontname = 'monospace', weight='semibold')
plt.xticks(rotation=60, ha='right', fontsize=10)
plt.yticks(fontsize=10)

labels = [item.get_text() for item in ax.get_xticklabels()]
labels = ['Analyze & Understand Data', ' Build & Run Data Infrastructure', ' Build ML Prototypes for New Areas', 'Build/Run ML Services', 'Experiment/Improve Existing ML Models',
         'Researching Advances in ML', 'None of these are Important', 'Other']
ax.set_xticklabels(labels)

plt.show()

- Well... With increasing the years of ml experience almost all responsibilities are getting greater. I thought things will get easier in future, oh... :(
- So the chart above confirms our assumption that the duties you cover/work increases with your knowledge.
- It also confirms that you can get new skills while working too.

# <div class="subsection_title">2.6.2 Employers Field of Industry</div>

- Since we looked at the roles and their responsibilities I feel like it's good time to check about who needs these jobs done and what industry they mainly work on.
- By looking at these we can get insights about more general picture of economy and what's the data science's role in it...

In [ ]:
count_dist(df=df, colname='In what industry is your current employer/contract (or your most recent employer if retired)?', max_idx=10, fontsize=20,
           rotation=60,
           title='Percentage of Employees by their Employers Field of Industry', palette=cust_color, shift=300, y_label= 'Field')

- First of all, before looking at these percentages above it's better to keep in your mind that more than one third of participants are already students or currently unemployed.
- It seems to me data related jobs are in earlier stages, I mean they still don't have widespread use from other fields. We can see that by looking top two fields:
    - Computers/Technology, well it's a broad term but it's already highly relevant with data related jobs. So it's expected these companies to hire people who works on data.
    - Secondly, there're decent number of participants who works in Academics/Education. That also shows data science still rising field, people still looking to find better use of data/ml and probably many discoveries to be made in future...
- Then comes Accounting/Finance which makes sense since in these fields companies already been gathering data for years and also it has practical uses like forecasting stocks in finance or finding anomalies in accounting records etc.
- The rest are rare but somewhat balanced. I see some fields where machine learning could get much more importance. But for now it seems like general trend is outsourcing these needs to more professional companies (Tech/Computer field).

# <div class="subsection_detail">2.6.2.1 Industry and Roles</div>

In [ ]:
# getting percentages and plotting them in styled heatmap,
z=df.groupby(['Select the title most similar to your current role (or most recent title if retired):',
    'In what industry is your current employer/contract (or your most recent employer if retired)?',
    
              ]).size().unstack().fillna(0).astype('int16')
fig, ax = plt.subplots(figsize=(16, 12))
sns.heatmap(z.apply(lambda x: x/x.sum(), axis=1), xticklabels=True, yticklabels=True, cmap='YlOrBr', annot=True, linewidths=0.005, linecolor='black', annot_kws={"fontsize":12}, fmt='.4f', cbar=False)
plt.title('Roles and Industry', fontname = 'monospace', weight='bold')
plt.xticks(fontsize=12,rotation=90)
plt.yticks(fontsize=12)
plt.ylabel("Employee Role", fontname = 'monospace', weight='semibold')
plt.xlabel("Employer Industry", fontname = 'monospace', weight='semibold')
plt.show()
del z

- The table above we can see distribution of roles by industry.
- Most of the roles are employed by computers/technology and acadamics/education industries as we seen from previous plot too. But let's take short look on rest.
- It seems accounting and finance industry employs more analyst positions.
- For government and public services, statisticians and database engineers are more popular than others.
- For some fields I hoped for little bit higher percentages and I believe they could take advantage of data much more:
    - Insurance / Risk Assesment: Where you could train model with lesser human bias.
    - Medical / Pharmaceutical: Especially computer vision could be life saver (literally) for early and accurate detections.
    - Retail / Online sales: Better customer segmentation or maybe stock estimation.
    - Shipping / Transportation: Where companies can optimize their workflow and increase their efficiency.
    
*Things looking good for you Robert, it might your options are limited at first sight but if you look closely future is promising. You might be working for tech startup or academia(if you want to continue after master's) and advance your career there and you might want to move bigger tech companies in future. Also you might get hired by a small company or small department of data science in bigger company. Options are countless as long as you add some skills that make you one step ahead of your competitors. And we discussed what you should do for that in previous chapter right? If you forgot you can just go back there and start reading again. I'll be waiting for you in next part...*


# <div class="section_title">2.7 Money, money, money...</div>

We looked at number of variables that might affect your career at data science and there is one more thing left, I know you have been waiting for this a while :) But I wanted to keep it this one for later stages because this variable is more of a result than a cause. So yeah, you probably guess already, it's the yearly compensation...

In [ ]:
count_dist(df=df, colname='What is your current yearly compensation (approximate $USD)?', max_idx=30, fontsize=10,
           rotation=45,
           title='Yearly Compensation of Surveyants', palette=cust_color, shift=120, y_label= 'Compensation in $')

- Ok, first of all, world is not that fair :( We can see huge gaps between yearly income groups and sadly it not unique to data field :/
- Anyways, when we look at the biggest income group,it's 0-999 USD range by far, but the rest is pretty hard to interpet, well at least for me, there might be many reasons:
    - First of all the income is correlated with many other things.
    - I believe biggest reason behind this is where the participant currently resides, every country has it's own income standards like:
        - GDP/C
        - Minimum Wage
        - Inflation / Interest rate
        - Exchange rates against USD etc.
    - Then comes more personal traits can affect a person's income, which we can interpet a little bit better with the data we have.
- There are many ways to clear effects to fix first issue with countries but I think that one out of scope of this notebook for now.
- Yet we can do some basic demographic analyses to explore it little bit further. 
- To do so we're going little bit back to demographics again.

*By the way I didn't forget about you Robert, you better keep learning the stuff I suggested you, as I finish with next part I believe you will be ready to get a job!*

# <div class="subsection_title">2.7.1 Income Demographics</div>
- Let's see how's the yearly income differs by some demographic features.
- When we look at the yearly compensation variable we can see it's binned (values are given between some ranges), so I'm going to take mean of these, It might be not the best approach but I believe that'll be enough to do the trick...

# <div class="subsection_detail">2.7.1.1 Country of Residence and Yearly Income</div>

In [ ]:
# splitting strings for yearly compensation, turning them to int and taking median
df[['min_w','max_w']]=df['What is your current yearly compensation (approximate $USD)?'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).str.replace('>', '', regex=False).str.split('-', expand = True)
df[['min_w','max_w']] = df[['min_w','max_w']].astype('float')
df['Mean_Compensation']=(df['min_w']+df['max_w'])/2 + 0.5
z =df.groupby('In which country do you currently reside?')['Mean_Compensation'].median()
z.sort_values(ascending=False, inplace=True)
fig, ax = plt.subplots(figsize=(16, 18))
sns.barplot(y=z.index, x=z.values, palette='YlOrBr_r', edgecolor='black', saturation = 2, linewidth = 0.5)

plt.title('Median Yearly Income by Countries', fontname = 'monospace', weight='bold')
plt.yticks(fontsize=10)
plt.xticks(np.arange(0, max(z.values)+1, 5000),fontsize=10)
plt.xlabel('Median YEarly Income in USD', fontname = 'monospace', weight='semibold')
plt.show()

- Before I plot the results I noticed high standard deviation for these variables so I decided to use median instead of mean. I believe this would give more robust results.
- As you can see from the table above there are huge gaps between countries, but again every country should be analysed by it's economic standards.
- The chart looks better than I expected at first, if you are little bit familiar with world economies and demographics you can notice there are similarities between them and this chart.
- You can see some countries are clustering toghether and they usually have similar economic parameters like money zone, gdp/c etc.
- I'm not going to analyse median income country by country since it would take another chapter itself, but feel free to do so;

*And lucky you Robert, you already residing in top pretty high median income country!*

# <div class="subsection_detail">2.7.1.2 Education and Yearly Income</div>

In [ ]:
# plotting boxplot
fig, ax = plt.subplots(figsize=(12, 6))
sns.boxplot(y="What is the highest level of formal education that you have attained or plan to attain within the next 2 years?", x="Mean_Compensation", data=df, linewidth = 1,
           palette=cust_color[::2], saturation=1.5)
plt.ylabel('Education', fontname = 'monospace', weight='semibold')
plt.yticks(fontsize=8)
plt.xticks(np.arange(0, max(df['Mean_Compensation'])+1, 50000),fontsize=8)
plt.xlabel('Yearly Income in USD', fontname = 'monospace', weight='semibold')
plt.title('Yearly Income by Education', fontname = 'monospace', weight='bold')
plt.show()

- Although there are some outliers we can see a clear pattern here: With increasing level of education median income by year increases too.

# <div class="subsection_detail">2.7.1.3 Role and Yearly Income</div>

In [ ]:
# plotting and styling boxplot
fig, ax = plt.subplots(figsize=(12, 6))
sns.boxplot(y="Select the title most similar to your current role (or most recent title if retired):", x="Mean_Compensation",
            data=df[(df["Select the title most similar to your current role (or most recent title if retired):"]!='Student')&
                   (df["Select the title most similar to your current role (or most recent title if retired):"]!='Currently not employed')], linewidth = 1,
           palette=cust_color, saturation=1.5)
plt.ylabel('Job Title', fontname = 'monospace', weight='semibold')
plt.yticks(fontsize=8)
plt.xticks(np.arange(0, max(df['Mean_Compensation'])+1, 50000),fontsize=8)
plt.xlabel('Yearly Income in USD', fontname = 'monospace', weight='semibold')
plt.title('Yearly Income by Role', fontname = 'monospace', weight='bold')
plt.show()

- Wait what?! Machine Learning Engineer has one of the lowest median income?
- I thought they earning little bit more than data scientists generally. There must be something wrong there, we need to dig that; but for now let's look at the rest...
- Seems like managerial roles has the highest income although having some large gaps between quantiles.
- Some roles like statisticians or data analysts are earning a little bit less than general averages.

*Hey Robert are you there? Sorry what?! You changed your mind over machine learning engineer after this chart? Hold on mate, as I said there must be something affecting this statistic, let me show you; hey listen listen... Go check the next chart!*

In [ ]:
# plotting median income for machine learning engineers by countrhy
z=df.groupby(['Select the title most similar to your current role (or most recent title if retired):','In which country do you currently reside?']).size()
z=z[z.index.isin(['Machine Learning Engineer'], level=0)].sort_values(ascending=False).reset_index().rename(columns={0: 'ML_Eng_Count'})[['In which country do you currently reside?','ML_Eng_Count']]
z.sort_values(ascending=False, inplace=True, by='ML_Eng_Count')
z=z.merge(df[df['Select the title most similar to your current role (or most recent title if retired):']=='Machine Learning Engineer'].groupby('In which country do you currently reside?')['Mean_Compensation'].median().reset_index(), on='In which country do you currently reside?')
fig, ax = plt.subplots(figsize=(12, 6))
sns.barplot(y=z.iloc[:10, 0], x=z.iloc[:10,1], palette=cust_color, edgecolor='black', saturation = 2, linewidth = 0.5)
for n, i in enumerate(z.iloc[:10, 0]):    
    ax.text(z.iloc[:10, 1][n]+68,
            n, #Y location
            s=f'Median MLE Income: {np.round(z.iloc[:10, 2][n]):.0f}', 
            va='center', 
            ha='right', 
            color='white', 
            fontsize=8,
            bbox=dict(boxstyle='round',facecolor=cust_color[-1], alpha=0.5))
plt.ylabel('Country of Residence', fontname = 'monospace', weight='semibold')
plt.yticks(fontsize=8)
plt.xticks(np.arange(0, max(z['ML_Eng_Count'])+1, 25),fontsize=8)
plt.xlabel('# Participants', fontname = 'monospace', weight='semibold')
plt.title('Median Income of Machine Learning Engineers for Top Countries', fontname = 'monospace', weight='bold')
plt.show()

- That's a relief, I think we found the reason behind the low median income for machine learning engineers globally.
- If you look at the countries where having the highest number of machine learning engineers you see most of them are placed lower ranks on the list in terms of median USD income.
- So high number of machine learning engineers from these countries are lowering the global median income statistics.
- You can see two exceptions on this list: USA and Japan, when you look closely their income values for data scientists and machine learning engineers you can see they're almost identical, meanwhile in countries like India data scientists almost making two times more than machine learning engineers, interesting...
- Is machine learning engineer title over-hyped in some countries and demand is low? That might be the case...

*You see Robert, there's no need to worry about that, you are interested in machine learning engineer role again? Ah nice, good lad!*

# <div class="subsection_detail">2.7.1.4 Industry and Yearly Income</div>

In [ ]:
# plotting and styling boxplot
fig, ax = plt.subplots(figsize=(12, 6))
sns.boxplot(y="In what industry is your current employer/contract (or your most recent employer if retired)?", x="Mean_Compensation",
            data=df[(df["Select the title most similar to your current role (or most recent title if retired):"]!='Student')&
                   (df["Select the title most similar to your current role (or most recent title if retired):"]!='Currently not employed')], linewidth = 1,
           palette=cust_color, saturation=1.5)
plt.ylabel('Industry', fontname = 'monospace', weight='semibold')
plt.yticks(fontsize=8)
plt.xticks(np.arange(0, max(df['Mean_Compensation'])+1, 50000),fontsize=8)
plt.xlabel('Yearly Income in USD', fontname = 'monospace', weight='semibold')
plt.title('Yearly Income by Industry', fontname = 'monospace', weight='bold')
plt.show()

- A saddening chart... While Military/Security/Defense Industry giving highest median wages the lowest median one is Academics/Education (If we don't count non-profit ones), I mean security is important in it's own way but I believe Academics should get more love for the future.
    - Personally I check high paying countries individually and the trend is similar, academics are at bottom in yearly compensation charts...
- The rest of the Industries seems balanced in their own clusters, especially when you include the country effect into the equation. But I'm not going to inspect all of these: Firsly I'm not that familiar with all of these industries to make assumptions and secondly that would take many charts to analyze.
- So I'd like to contine by taking picture of the industry in regards to mean yearly compensation, so let's continue with that.

In [ ]:
# plotting and styling boxplot
fig, ax = plt.subplots(figsize=(12, 6))
sns.boxplot(y="What is the size of the company where you are employed?", x="Mean_Compensation",
            data=df[(df["Select the title most similar to your current role (or most recent title if retired):"]!='Student')&
                   (df["Select the title most similar to your current role (or most recent title if retired):"]!='Currently not employed')], linewidth = 1,
           palette=cust_color, saturation=1.5)
plt.ylabel('# of Employees', fontname = 'monospace', weight='semibold')
plt.yticks(fontsize=8)
plt.xticks(np.arange(0, max(df['Mean_Compensation'])+1, 50000),fontsize=8)
plt.xlabel('Yearly Income in USD', fontname = 'monospace', weight='semibold')
plt.title('Yearly Income by Company Size', fontname = 'monospace', weight='bold')
plt.show()

- The trend seems obvious, the bigger the company gets the more wage they pay.
- You could interpret the size of a company with level of corporateness where corporate firms having higher benefits, more standardized wages etc.

In [ ]:
# plotting and styling boxplot
fig, ax = plt.subplots(figsize=(12, 6))
sns.boxplot(y="Does your current employer incorporate machine learning methods into their business?", x="Mean_Compensation",
            data=df[(df["Select the title most similar to your current role (or most recent title if retired):"]!='Student')&
                   (df["Select the title most similar to your current role (or most recent title if retired):"]!='Currently not employed')], linewidth = 1,
           palette=cust_color, saturation=1.5)
plt.ylabel('ML Incorporation', fontname = 'monospace', weight='semibold')
plt.yticks(fontsize=8)
plt.xticks(np.arange(0, max(df['Mean_Compensation'])+1, 50000),fontsize=8)
plt.xlabel('Yearly Income in USD', fontname = 'monospace', weight='semibold')
plt.title('Yearly Income by Company ML Incorporation', fontname = 'monospace', weight='bold')
plt.show()

- Again a clear trend...
- So, the companies where incorporated ML methods into their business are paying more for the data workers. This could be due to some reasons, like:
    - The company understands the value of data and willing to pay even more for right person.
    - After using ML methods for a while the requirements are getting greater and complex so companies trying to get more skillful employees.
    - The added-value generated by ML is significant so data department getting higher budgets inside the company.
    - The demand is high for people who can work with ML stuff but the qualified workforce is low.
    

In [ ]:
# plotting and styling boxplot
fig, ax = plt.subplots(figsize=(12, 6))
sns.boxplot(y="Approximately how many individuals are responsible for data science workloads at your place of business?", x="Mean_Compensation",
            data=df[(df["Select the title most similar to your current role (or most recent title if retired):"]!='Student')&
                   (df["Select the title most similar to your current role (or most recent title if retired):"]!='Currently not employed')], linewidth = 1,
           palette=cust_color, saturation=1.5)
plt.ylabel('# Employees', fontname = 'monospace', weight='semibold')
plt.yticks(fontsize=8)
plt.xticks(np.arange(0, max(df['Mean_Compensation'])+1, 50000),fontsize=8)
plt.xlabel('Yearly Income in USD', fontname = 'monospace', weight='semibold')
plt.title('Yearly Income by Number of People in Data Science Workloads', fontname = 'monospace', weight='bold')
plt.show()

- This one confirms our assumptions in previous charts:
    - The need for people who can work with data increases how much the company is willing to pay for it's employees.

# <div class="subsection_title">2.7.2 Some Cluster-Fun</div>

- Here I'm going to reduce dimension of the data and try to find meaningful clusters, it'a unsupervised learning method might help us to understand more about the data itself...

In [ ]:
# taking columns as list we inspected above for future use
cat_cols = ['What is your age (# years)?', 'What is your gender?', 'In which country do you currently reside?',
            'What is the highest level of formal education that you have attained or plan to attain within the next 2 years?',
            'Select the title most similar to your current role (or most recent title if retired):', 'For how many years have you been writing code and/or programming?',
            'For how many years have you used machine learning methods?',
            'In what industry is your current employer/contract (or your most recent employer if retired)?',
            'What is the size of the company where you are employed?',
            'Approximately how many individuals are responsible for data science workloads at your place of business?',
            'Does your current employer incorporate machine learning methods into their business?',
            'What is your current yearly compensation (approximate $USD)?'
           ]


multi_cols = [
 'What programming languages do you use on a regular basis? Python',
 'What programming languages do you use on a regular basis? R',
 'What programming languages do you use on a regular basis? SQL',
 'What programming languages do you use on a regular basis? C',
 'What programming languages do you use on a regular basis? C++',
 'What programming languages do you use on a regular basis? Java',
 'What programming languages do you use on a regular basis? Javascript',
 'What programming languages do you use on a regular basis? Julia',
 'What programming languages do you use on a regular basis? Swift',
 'What programming languages do you use on a regular basis? Bash',
 'What programming languages do you use on a regular basis? MATLAB',
 'What programming languages do you use on a regular basis? None',
 'What programming languages do you use on a regular basis? Other',
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? Jupyter (JupyterLab, Jupyter Notebooks, etc)",
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? RStudio",
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? Visual Studio",
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? Visual Studio Code (VSCode)",
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? PyCharm",
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? Spyder",
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? Notepad++",
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? Sublime Text",
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? Vim / Emacs",
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? MATLAB",
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? Jupyter Notebook",
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? None",
 "Which of the following integrated development environments (IDE's) do you use on a regular basis? Other",
 'Which of the following hosted notebook products do you use on a regular basis? Kaggle Notebooks',
 'Which of the following hosted notebook products do you use on a regular basis? Colab Notebooks',
 'Which of the following hosted notebook products do you use on a regular basis? Azure Notebooks',
 'Which of the following hosted notebook products do you use on a regular basis? Paperspace / Gradient',
 'Which of the following hosted notebook products do you use on a regular basis? Binder / JupyterHub',
 'Which of the following hosted notebook products do you use on a regular basis? Code Ocean',
 'Which of the following hosted notebook products do you use on a regular basis? IBM Watson Studio',
 'Which of the following hosted notebook products do you use on a regular basis? Amazon Sagemaker Studio Notebooks',
 'Which of the following hosted notebook products do you use on a regular basis? Amazon EMR Notebooks',
 'Which of the following hosted notebook products do you use on a regular basis? Google Cloud Notebooks (AI Platform / Vertex AI)',
 'Which of the following hosted notebook products do you use on a regular basis? Google Cloud Datalab',
 'Which of the following hosted notebook products do you use on a regular basis? Databricks Collaborative Notebooks',
 'Which of the following hosted notebook products do you use on a regular basis? Zeppelin / Zepl Notebooks',
 'Which of the following hosted notebook products do you use on a regular basis? Deepnote Notebooks',
 'Which of the following hosted notebook products do you use on a regular basis? Observable Notebooks',
 'Which of the following hosted notebook products do you use on a regular basis? None',
 'Which of the following hosted notebook products do you use on a regular basis? Other',
 'What data visualization libraries or tools do you use on a regular basis? Matplotlib',
 'What data visualization libraries or tools do you use on a regular basis? Seaborn',
 'What data visualization libraries or tools do you use on a regular basis? Plotly / Plotly Express',
 'What data visualization libraries or tools do you use on a regular basis? Ggplot / ggplot2',
 'What data visualization libraries or tools do you use on a regular basis? Shiny',
 'What data visualization libraries or tools do you use on a regular basis? D3 js',
 'What data visualization libraries or tools do you use on a regular basis? Altair',
 'What data visualization libraries or tools do you use on a regular basis? Bokeh',
 'What data visualization libraries or tools do you use on a regular basis? Geoplotlib',
 'What data visualization libraries or tools do you use on a regular basis? Leaflet / Folium',
 'What data visualization libraries or tools do you use on a regular basis? None',
 'What data visualization libraries or tools do you use on a regular basis? Other',
 'Which of the following machine learning frameworks do you use on a regular basis? Scikit-learn',
 'Which of the following machine learning frameworks do you use on a regular basis? TensorFlow',
 'Which of the following machine learning frameworks do you use on a regular basis? Keras',
 'Which of the following machine learning frameworks do you use on a regular basis? PyTorch',
 'Which of the following machine learning frameworks do you use on a regular basis? Fast.ai',
 'Which of the following machine learning frameworks do you use on a regular basis? MXNet',
 'Which of the following machine learning frameworks do you use on a regular basis? Xgboost',
 'Which of the following machine learning frameworks do you use on a regular basis? LightGBM',
 'Which of the following machine learning frameworks do you use on a regular basis? CatBoost',
 'Which of the following machine learning frameworks do you use on a regular basis? Prophet',
 'Which of the following machine learning frameworks do you use on a regular basis? H2O 3',
 'Which of the following machine learning frameworks do you use on a regular basis? Caret',
 'Which of the following machine learning frameworks do you use on a regular basis? Tidymodels',
 'Which of the following machine learning frameworks do you use on a regular basis? JAX',
 'Which of the following machine learning frameworks do you use on a regular basis? PyTorch Lightning',
 'Which of the following machine learning frameworks do you use on a regular basis? Huggingface',
 'Which of the following machine learning frameworks do you use on a regular basis? None',
 'Which of the following machine learning frameworks do you use on a regular basis? Other',
 'Which of the following ML algorithms do you use on a regular basis? Linear or Logistic Regression',
 'Which of the following ML algorithms do you use on a regular basis? Decision Trees or Random Forests',
 'Which of the following ML algorithms do you use on a regular basis? Gradient Boosting Machines (xgboost, lightgbm, etc)',
 'Which of the following ML algorithms do you use on a regular basis? Bayesian Approaches',
 'Which of the following ML algorithms do you use on a regular basis? Evolutionary Approaches',
 'Which of the following ML algorithms do you use on a regular basis? Dense Neural Networks (MLPs, etc)',
 'Which of the following ML algorithms do you use on a regular basis? Convolutional Neural Networks',
 'Which of the following ML algorithms do you use on a regular basis? Generative Adversarial Networks',
 'Which of the following ML algorithms do you use on a regular basis? Recurrent Neural Networks',
 'Which of the following ML algorithms do you use on a regular basis? Transformer Networks (BERT, gpt-3, etc)',
 'Which of the following ML algorithms do you use on a regular basis? None',
 'Which of the following ML algorithms do you use on a regular basis? Other',
 'Select any activities that make up an important part of your role at work: Analyze and understand data to influence product or business decisions',
 'Select any activities that make up an important part of your role at work: Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data',
 'Select any activities that make up an important part of your role at work: Build prototypes to explore applying machine learning to new areas',
 'Select any activities that make up an important part of your role at work: Build and/or run a machine learning service that operationally improves my product or workflows',
 'Select any activities that make up an important part of your role at work: Experimentation and iteration to improve existing ML models',
 'Select any activities that make up an important part of your role at work: Do research that advances the state of the art of machine learning',
 'Select any activities that make up an important part of your role at work: None of these activities are an important part of my role at work',
 'Select any activities that make up an important part of your role at work: Other'

]

df_ml = df.copy()


le = LabelEncoder()

# creating the "robert" :))

robert = pd.DataFrame([['22-24', 'Man', 'United States of America', "Bachelor’s degree", 'Student', '< 1 years','I do not use machine learning methods',
                             np.nan, np.nan, np.nan, np.nan, np.nan]],columns = df_ml[cat_cols].columns)

# label encoding categorical columns

for col in cat_cols:
    df_ml[col] = df_ml[col].fillna('Unk')
    df_ml[col] = le.fit_transform(df_ml[col].astype(str))
    robert[col] = robert[col].fillna('Unk')
    robert[col] = le.transform(robert[col].astype(str))
    
# creating the pipeline for dimension reduction
    
pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(2))])
df_r = pipe.fit_transform(df_ml[cat_cols])
robert_r = pipe.transform(robert)

- I'm going to use some of the data we analysed visually before while excluding the ones we didn't analyse properly, but firstly we going to reduce dimension of that data to 2D space using "Principal Component Analysis" so we can plot it easily and make it human readable :)
- Then we're going to try various number of clusters to see intertia values (basically mean distance of instances to the cluster centroid), of course increasing k naturally decreases the inertia, so we looking for the "Elbow" where inertia decrease slows down sharply.

In [ ]:
# finding best number of clusters
# source https://github.com/ageron/handson-ml2/blob/master/09_unsupervised_learning.ipynb

kmeans_per_k = [KMeans(n_clusters=k, random_state=42, max_iter=1000, tol=1e-5).fit(df_r)
                for k in range(1, 10)]
inertias = [model.inertia_ for model in kmeans_per_k]

plt.figure(figsize=(6, 3))
sns.lineplot(range(1, 10), inertias, color=cust_color[0], linewidth = 1.5)
plt.xlabel("k", fontsize=15)
plt.ylabel("Inertia", fontsize=15)
plt.annotate('Elbow',
             xy=(3, inertias[2]),
             xytext=(0.55, 0.55),
             textcoords='figure fraction',
             fontsize=16,
             arrowprops=dict(facecolor='black', shrink=0.05)
            )
plt.title('Inertias and n_clusters', fontname = 'monospace', weight='bold')
plt.show()

- So our k can be 2 or 3 but 3 seems little bit healthier. That means we're going to cluster all instances(participants) into 3 groups.
- Let's plot that to see if it makes any sense...

In [ ]:
# plotting 3 kmean clusters and clusters by role
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(df_r)

fig, ax = plt.subplots(1,2,figsize=(16, 9))
sns.scatterplot(df_r[:,0], df_r[:,1], hue=clusters, palette='Set1', ax=ax[0])
sns.scatterplot(robert_r[:,0], robert_r[:,1], color='red',s=150,marker='X', ax=ax[0])
sns.scatterplot(df_r[:,0], df_r[:,1], hue=df8['Select the title most similar to your current role (or most recent title if retired):'], palette='Set1', ax=ax[1], alpha=0.8)
sns.scatterplot(robert_r[:,0], robert_r[:,1], color='red',s=150,marker='X', ax=ax[1])

# setting an arrow and point for robert :)

ax[0].annotate(
"Here's Robert!", xy=(robert_r[0][0], robert_r[0][1]),xytext=(robert_r[0][0]+0.2, robert_r[0][1]+1.7),
horizontalalignment="center",
arrowprops=dict(lw=2, color='black', shrink=0.1), fontsize=12
)




plt.setp(ax[1].get_legend().get_texts(), fontsize='5') 
ax[1].get_legend().set_title("Role")
ax[0].get_legend().set_title("Cluster")
plt.setp(ax[1].get_legend().get_title(), fontsize='10') 
plt.suptitle('2D Representations of Clusters ', fontname = 'monospace', weight='bold')
ax[0].set_title('K-Means Clusters', fontname = 'monospace', weight='semibold')
ax[1].set_title('Role Clusters', fontname = 'monospace', weight='semibold')
plt.show()

- Ok... Not great, not terrible, I can see one cluster diverges a little bit more than others meanwhile cluster 0 and 2 is just one big blob :)
- When we plot same data with coloring instances based on their role we can see two roles are fitting pretty good on cluster 1 on left:
    - Students and Currently Unemployed ones are mainly sitting in the cluster 1,
    - Other clusters are just chaotic :) It might be due to data itself or 2D might not be enough to plot that for human eyes...
    
*Hey Robert! You see yourself there? Wave at us! Here you are! So you are fine fit for cluster 1, this is your new team, meet your friends! Cluster 1 mainly consists of students and unemployed people so cluster 1 is for people who starting their data science career recently. If you want to compare yourself with other people, using this tecnique can help you with reducing your scope so you can make fine grained analyses. Anyways as you can see you are not alone at all and with the takeaways from this notebook you can make rapid advances to other clusters you want to be in :)*

- Since we reduced the dimensions let's check what's the percentage of variance explained by principal components:

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()),('pca', PCA())])
pipe.fit(df_ml[cat_cols])
pca_samples = pipe.transform(df_ml[cat_cols])

# explaining variance ratio:

fig, ax = plt.subplots(figsize=(14, 5))
plt.plot(range(df_ml[cat_cols].shape[1]), pipe.named_steps['pca'].explained_variance_ratio_.cumsum(), linestyle='--', drawstyle='steps-mid', color=cust_color[-1],
         label='Cumulative Explained Variance', linewidth = 1.5)
sns.barplot(np.arange(1,df_ml[cat_cols].shape[1]+1), pipe.named_steps['pca'].explained_variance_ratio_, alpha=0.85, color=cust_color[0],
            label='Individual Explained Variance', edgecolor='black', saturation = 2, linewidth = 0.5)

plt.ylabel('Explained Variance Ratio', fontsize = 14, fontname = 'monospace', weight='semibold')
plt.xlabel('Number of Principal Components', fontsize = 14, fontname = 'monospace', weight='semibold')
ax.set_title('Explained Variance', fontsize = 20, fontname = 'monospace', weight='bold')
plt.legend(loc='center right', fontsize = 13);

- Oh this explains a lot... Well I mean actually it doesn't explain the variance a lot :) We used first two principal components to visualize the and it only explains 40% of the variance.
- Let's take a look one more thing about dimension reduction then...

In [ ]:
# plotting principal components
pipe = Pipeline([ ('pca', PCA(5))])
pipe.fit(df_ml[cat_cols])
pca_samples = pipe.transform(df_ml[cat_cols])

total_var = pipe.named_steps['pca'].explained_variance_ratio_.sum() * 100

labels = {str(i): f'PC {i+1}' for i in range(5)}
labels['color'] = 'Role'

fig = px.scatter_matrix(
    pca_samples,
    color=df8['Select the title most similar to your current role (or most recent title if retired):'],
    dimensions=range(5),
    labels=labels,
    title=f'Principal Components',
    opacity=0.8)
fig.update_traces(diagonal_visible=False)
fig.show()

- Well well, take a look at this one...
- Looks like 4th principal component represents the Role feature really well, look how clustered the instances are! They look like cute rainbows :)

*Isn't it fun to find hidden stuff inside the data Robert? You having fun there? As I told you before, with little bit knowledge of Sklearn and visualization tools you can find new ways to gain insights. Wait you already clustered Iris dataset? Nice one Robert! It's a great choice to learn about clustering, well done!*

# <div class="top_section">Something Personal...</div>

Untill now I used the actual data itself. I wonder how's the text written by me along this notebook, what points I kept dwell on, what words I over used :/ When I turn back and check the notebook I found out I wrote loads of things, I just crawled them and extracted it to a single .txt file, I think its worth to visualize it :) Let's see:

In [ ]:
# importing packages and loading text
from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image

f = open("../input/mynotebooktext/mytext.txt", "r")
text = f.read()
text = re.sub(r'[^\w\s]','',text)
text = "".join(token.lower() for token in text)
texts = text.split()


# getting tokens
texts = " ".join(token for token in texts)

font_path = "../input/mynotebooktext/acetone_font.otf"

# generating wordcloud
wordcloud = WordCloud(font_path=font_path,
                      max_words=1500,
                      max_font_size=500, random_state=42,
                      width=2000, height=1000,
                      colormap = "gist_stern")
wordcloud.generate(texts)

# plotting
plt.figure(figsize = (24, 13))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title('Most Common Words in Markdowns Through the Notebook', fontname = 'monospace', weight='semibold', fontsize = 40)
plt.show()

Ok, maybe I over used some words like "see, one etc." usually they coming from the parts where I point out a plot or finding like I just did above (facepalm)... If we count these as stopwords then we can see some more important words like:
- "Data",
- "Data Science",
- "Data Scientist",
- "Machine Learning" or "ML",
- "Machine Learning Engineer",
- And many more related stuff...

And of course the **Robert**, we couldn't done it without him :)

Anyways it's time to wrap up things...

# <div class="top_section">Endings</div>


<img src="https://i.imgur.com/NPSdyrx.png" style="height: 180; width: 100%; object-fit: cover;"/>

[Source: Know Your Meme](https://knowyourmeme.com/memes/pepe-silvia)

Robert has been getting quite number of suggestions for a while, we could only give advices but the rest is was on him. Let's see how he handled the process; did he managed to pull it off? Here are the possible outcomes:

# <div class="subsection_title">The Bad Ending...</div>

<img src='https://image.freepik.com/free-vector/depression-concept-illustration_114360-3915.jpg' width=600>
<div align="center"><font size="1">Source: <a href="https://www.freepik.com/free-vector/depression-concept-illustration_10768176.htm#page=1&query=sad&position=2&from_view=author">Sad by "Stories" - www.freepik.com</a></font></div>  

*Robert barely listened the story of data, he didn't do much to improve his data science skills, he felt like he had enough knowledge to jump right into industry. He applied for many many jobs, he didn't check for the requirements, he just "brute force" his way to the field. After hundreds of applications he either failed interviews or didn't get reply at all. At the end he found a "Data Science" internship where he does data entry tasks all day, nothing near the data science career he dreamt for...*

# <div class="subsection_title">The Neutral Ending...</div>

<img src='https://image.freepik.com/free-vector/stress-concept-illustration_114360-2110.jpg' width=600>
<div align="center"><font size="1">Source: <a href="https://www.freepik.com/free-vector/stress-concept-illustration_8392598.htm#page=1&query=stressed&position=3&from_view=author">Streessed by "Stories" - www.freepik.com</a></font></div>  

Robert listened and took some notes from the story of data. He learnt some basics of libraries here and there, but he made an one fatal mistake: He went too fast; he started learning one library and jumped other one in no time before getting hang of the previous library. For example he learnt sklearn's Logistic Regression only in practice and went for Neural Networks without understanding how's the Logistic Regression or Perceptron works, now he's trying to work on Convolutional Neural Networks. Luckily after some job applications he found a "Machine Learning Engineer" role, he received a contract and he signed it. But things are not going in Robert's way, he get tasks from his work where he slightly understands the concept and after stressful hours of working he barely saves the day for next battle. He fears he's gonna get fired one day he also gets stressed about his job and thinking to quit in order to re-learn stuff he went over before...

# <div class="subsection_title">The Happy Ending...</div>

<img src='https://image.freepik.com/free-vector/personal-goals-concept-illustration_114360-5626.jpg' width=600>
<div align="center"><font size="1">Source: <a href="https://www.freepik.com/free-vector/personal-goals-concept-illustration_13416257.htm#page=1&query=success&position=4&from_view=author">Success by "Stories" - www.freepik.com</a></font></div>  

Robert carefully listened and inspected the story of data. He methodically planned his roadmap, instead of aiming for unattainable targets and rapid jumps he structured his learning path. He didn't move to the next step before he gets comfortable with previous concept. He learned libraries well both in practice and theory, maybe he didn't have chance to work on "Cool" stuff where they attract big hype lately but he knows how to learn them and he has broad idea about how do they work. For example he doesn't know how to work with transformer language models yet, but he is really comfortable working with tf-idf or count vectorizers. He also learned about word embeddings so he does know about importance contextuality of a word in a sentence too. With this way he can understand shortcomings of old methods and how're newer methods fixing them. He carefully selected his applications, he went for job adverts where the requirements are nicely fits to his profile. After some time he got his first job offer from a decent company where the company knows about what they need. His job title is "Junior Data Scientist". His daily activites are already specified, he's happy to work, if he doesn't know the concept or encounters a problem he knows he has decent mentors to help him. Maybe some people doesn't find his job title "Fancy" (because of Jr. stuff) yet, but he knows he's on right path and adding new skills regulary to advance in his career...

# <div class="section_title">Conclusions...</div>

Here we are, the closing part. I spent decent amount of my free time from work to finish this one, but I think it's worth it, I learnt a lot while preparing this notebook myself! I just want to thank <span style="color:#00FFFF;"> kaggle </span> for the enjoyable dataset to work on. Also would like to thank whole <span style="color:#00FFFF;"> kaggle </span> community for their support and the contributions, I learnt lots of tricks on <span style="color:#00FFFF;"> kaggle </span> from countless number of public notebooks in past, I hope you find something in this one too!

I tried to explain most of the charts as simple as possible, some parts might be little bit confusing for beginners but I believe there is a limited number of them and again I tried to explain them as clear as possible. But feel free to ask me personally if you have a question about any of them.

I might have used many plots, it can get tiresome to interpret them in one go at some point, you don't have to rush; you can take your time or just choose the ones where you're interested in, I added chapters for this purpose. I also put emphasis on explain them by markdowns, therefore they can be easier to read.

Throughout the notebook I endeavor to stay positive, although I have seen some red flags around some statistics (like gender gap, income gap or unbalanced role distributions, role hypes etc.) but I tried to encourage beginners, because I know it can be hard to get into data science field these days...

I aimed to combine my personal experiences/opinions with my interpretations from the data, some might find them subjective but I tried to stay objective as much as I can while trying to see positive side of the things...

**About Robert...**

Robert was a randomly generated character, I choose his attributes randomly (with weights) from the data itself. He was a fitting character for most of the participants, so while guiding him through the notebook I tried to help beginners who's in similar shoes.

About the suggestions, I know there isn't a single absolute right way to get successful in your data science career but as I said I tried to share my personal opinions/experiences with the help of data, if you find some of these suggestions not fitting for you, don't worry about it, I'm pretty sure there are many many more ways to get your goals done...

My main point here is you need to make an effort to do something differs you from the others in this harsh competitive industry, you can see things are not fair most of the time in our world, but if you motivated enough I'm pretty sure you'll get rewarded at the end of your journey. Some of you might find this too optimistic (Well I get pessimist too most of the time :/ ) but with methodical and planned approach I think it's achieveable...

Instead of giving a single outcome at the end I choose to give you three possible endings. I leave what outcome Robert is going to get for your imagination but I hope he's happy :)

Lastly I want to add that I'm still in early parts of my journey myself, there's lot to learn and share in future, but humbly I wanted to share what I think on this subject. I love sharing spirit of this community and I'm really glad to be part of it...

**I hope you enjoyed this notebook and found something useful for you. Even one person finds this work useful, that would make me more than happy! Thank you all for reading :)**